# 1692 Hard 1692 Count Ways to Distribute Candies

In [ ]:
# Time:  O(n * k)
# Space: O(k)

class Solution(object):
    def waysToDistribute(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: int
        """
        MOD = 10**9+7
        dp = [1]*k
        for i in xrange(1, n):
            for j in reversed(xrange(1, min(i, k))):
                dp[j] = ((j+1)*dp[j] + dp[j-1]) % MOD
        return dp[k-1]

# 1697 Hard 1697 Checking Existence of Edge Length Limited Paths

In [ ]:
# Time:  O(nlogn + mlogm + n * Î±(n)) = O(nlogn + mlogm)
# Space: O(n)

class UnionFind(object):  # Time: O(n * Î±(n)), Space: O(n)
    def __init__(self, n):
        self.set = range(n)
        self.rank = [0]*n

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y):
        x_root, y_root = map(self.find_set, (x, y))
        if x_root == y_root:
            return False
        if self.rank[x_root] < self.rank[y_root]:  # union by rank
            self.set[x_root] = y_root
        elif self.rank[x_root] > self.rank[y_root]:
            self.set[y_root] = x_root
        else:
            self.set[y_root] = x_root
            self.rank[x_root] += 1
        return True


class Solution(object):
    def distanceLimitedPathsExist(self, n, edgeList, queries):
        """
        :type n: int
        :type edgeList: List[List[int]]
        :type queries: List[List[int]]
        :rtype: List[bool]
        """
        for i, q in enumerate(queries):
            q.append(i)
        edgeList.sort(key=lambda x: x[2])
        queries.sort(key=lambda x: x[2])
        
        union_find = UnionFind(n)
        result = [False]*len(queries)
        curr = 0
        for u, v, w, i in queries: 
            while curr < len(edgeList) and edgeList[curr][2] < w: 
                union_find.union_set(edgeList[curr][0], edgeList[curr][1])
                curr += 1
            result[i] = union_find.find_set(u) == union_find.find_set(v)
        return result 

# 1703 Hard 1703 Minimum Adjacent Swaps for K Consecutive Ones

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def minMoves(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        def moves(i, j):
            return prefix[j+1]-prefix[i]

        idxs = [i for i, x in enumerate(nums) if x]
        prefix = [0]*(len(idxs)+1)
        for i in xrange(len(idxs)):
            prefix[i+1] = prefix[i]+idxs[i]
        result = float("inf")
        for i in xrange(len(idxs)-k+1):
            result = min(result, -moves(i, i+k//2-1) + moves(i+(k+1)//2, i+k-1))  # take each i+k//2 as median, find min dist to median
        result -= (k//2)*((k+1)//2)  # rollback extra moves to the expected positions
        return result

# 1707 Hard 1707 Maximum XOR With an Element From Array

In [ ]:
# Time:  O(nlogn + mlogm + nlogk + mlogk), k is max(max(nums), max(xi))
# Space: O(nlogk)

class Trie(object):
    def __init__(self, bit_length):
        self.__root = {}
        self.__bit_length = bit_length
        
    def insert(self, num):
        node = self.__root
        for i in reversed(xrange(self.__bit_length)):
            curr = (num>>i) & 1
            if curr not in node:
                node[curr] = {}
            node = node[curr]
                
    def query(self, num):
        if not self.__root: 
            return -1
        node, result = self.__root, 0
        for i in reversed(xrange(self.__bit_length)):
            curr = (num>>i) & 1
            if 1^curr in node:
                node = node[1^curr]
                result |= 1<<i
            else:
                node = node[curr]
        return result


class Solution(object):
    def maximizeXor(self, nums, queries):
        """
        :type nums: List[int]
        :type queries: List[List[int]]
        :rtype: List[int]
        """
        nums.sort()
        max_val = max(nums[-1], max(queries, key=lambda x: x[0])[0])
        queries = sorted(enumerate(queries), key=lambda x: x[1][1])        
        trie = Trie(max_val.bit_length())
        result = [-1]*len(queries)
        j = 0
        for i, (x, m) in queries:
            while j < len(nums) and nums[j] <= m:
                trie.insert(nums[j])
                j += 1
            result[i] = trie.query(x)
        return result

# 1713 Hard 1713 Minimum Operations to Make a Subsequence

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import bisect


class Solution(object):
    def minOperations(self, target, arr):
        """
        :type target: List[int]
        :type arr: List[int]
        :rtype: int
        """
        lookup = {x:i for i, x in enumerate(target)}
        lis = []
        for x in arr:
            if x not in lookup:
                continue
            i = bisect.bisect_left(lis, lookup[x])
            if i == len(lis):
                lis.append(lookup[x])
            else:
                lis[i] = lookup[x]
        return len(target)-len(lis)
    
    
# Range Maximum Query
class SegmentTree(object):  # 0-based index
    def __init__(self, N,
                 build_fn=lambda x, y: [y]*(2*x),
                 query_fn=lambda x, y: y if x is None else max(x, y),  # (lambda x, y: y if x is None else min(x, y))
                 update_fn=lambda x, y: y,
                 default_val=0):
        self.N = N
        self.H = (N-1).bit_length()
        self.query_fn = query_fn
        self.update_fn = update_fn
        self.default_val = default_val
        self.tree = build_fn(N, default_val)
        self.lazy = [None]*N

    def __apply(self, x, val):
        self.tree[x] = self.update_fn(self.tree[x], val)
        if x < self.N:
            self.lazy[x] = self.update_fn(self.lazy[x], val)

    def update(self, L, R, h):  # Time: O(logN), Space: O(N)
        def pull(x):
            while x > 1:
                x //= 2
                self.tree[x] = self.query_fn(self.tree[x*2], self.tree[x*2+1])
                if self.lazy[x] is not None:
                    self.tree[x] = self.update_fn(self.tree[x], self.lazy[x])

        L += self.N
        R += self.N
        L0, R0 = L, R
        while L <= R:
            if L & 1:  # is right child
                self.__apply(L, h) 
                L += 1
            if R & 1 == 0:  # is left child
                self.__apply(R, h)
                R -= 1
            L //= 2
            R //= 2
        pull(L0)
        pull(R0)

    def query(self, L, R):  # Time: O(logN), Space: O(N)
        def push(x):
            n = 2**self.H
            while n != 1:
                y = x // n
                if self.lazy[y] is not None:
                    self.__apply(y*2, self.lazy[y])
                    self.__apply(y*2 + 1, self.lazy[y])
                    self.lazy[y] = None
                n //= 2

        result = None
        if L > R:
            return result

        L += self.N
        R += self.N
        push(L)
        push(R)
        while L <= R:
            if L & 1:  # is right child
                result = self.query_fn(result, self.tree[L])
                L += 1
            if R & 1 == 0:  # is left child
                result = self.query_fn(result, self.tree[R])
                R -= 1
            L //= 2
            R //= 2
        return result
    
    def __str__(self):
        showList = []
        for i in xrange(self.N):
            showList.append(self.query(i, i))
        return ",".join(map(str, showList))


# Time:  O(nlogn)
# Space: O(n)
# segment tree solution
class Solution2(object):
    def minOperations(self, target, arr):
        """
        :type target: List[int]
        :type arr: List[int]
        :rtype: int
        """
        lookup = {x:i for i, x in enumerate(target)}
        st = SegmentTree(len(lookup))
        for x in arr:
            if x not in lookup:
                continue
            st.update(lookup[x], lookup[x], st.query(0, lookup[x]-1)+1 if lookup[x] >= 1 else 1)
        return len(target)-(st.query(0, len(lookup)-1) if len(lookup) >= 1 else 0)

# 1719 Hard 1719 Number Of Ways To Reconstruct A Tree

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import collections


class Solution(object):
    def checkWays(self, pairs):
        """
        :type pairs: List[List[int]]
        :rtype: int
        """
        adj = collections.defaultdict(set)
        for x, y in pairs:
            adj[x].add(y)
            adj[y].add(x)
        n, mul = len(adj), False
        lookup = set()
        for node in sorted(adj.iterkeys(), key=lambda i: len(adj[i]), reverse=True):
            lookup.add(node)
            parent = 0
            for x in adj[node]:
                if x not in lookup:
                    continue
                if parent == 0 or len(adj[x]) < len(adj[parent]):
                    parent = x
            if parent:
                if any(True for x in adj[node] if x != parent and x not in adj[parent]):
                    return 0
                mul |= len(adj[parent]) == len(adj[node])
            elif len(adj[node]) != n-1:
                return 0
        return 1 + mul

# 1723 Hard 1723 Find Minimum Time to Finish All Jobs

In [ ]:
# Time:  O(k^n * logr), the real complexity shoud be much less, but hard to analyze
# Space: O(n + k)

class Solution(object):
    def minimumTimeRequired(self, jobs, k):
        """
        :type jobs: List[int]
        :type k: int
        :rtype: int
        """
        def backtracking(jobs, i, cap, counts):
            if i == len(jobs):
                return True
            for j in xrange(len(counts)):
                if counts[j]+jobs[i] <= cap:
                    counts[j] += jobs[i]
                    if backtracking(jobs, i+1, cap, counts):
                        return True
                    counts[j] -= jobs[i]
                if counts[j] == 0:
                    break
            return False

        jobs.sort(reverse=True)
        left, right = max(jobs), sum(jobs)
        while left <= right:
            mid = left + (right-left)//2
            if backtracking(jobs, 0, mid, [0]*k):
                right = mid-1
            else:
                left = mid+1
        return left


# Time:  O(k * k^n), the real complexity shoud be less, but hard to analyze
# Space: O(n + k)
class Solution2(object):
    def minimumTimeRequired(self, jobs, k):
        """
        :type jobs: List[int]
        :type k: int
        :rtype: int
        """
        def backtracking(jobs, i, counts, result):
            if i == len(jobs):
                result[0] = min(result[0], max(counts))
                return
            for j in xrange(len(counts)):
                if counts[j]+jobs[i] <= result[0]:
                    counts[j] += jobs[i]
                    backtracking(jobs, i+1, counts, result)
                    counts[j] -= jobs[i]
                if counts[j] == 0:
                    break

        jobs.sort(reverse=False)
        result = [sum(jobs)]
        backtracking(jobs, 0, [0]*k, result)
        return result[0]

# 1724 Hard 1724 Checking Existence of Edge Length Limited Paths II

In [ ]:
# Time:  ctor:  O(mlogm + m * Î±(n) + nlogn) ~= O(mlogm + nlogn)
#        query: O(Î±(n) + logn) ~= O(logn)
# Space: O(nlogn + m)

from functools import partial

# Template:
# https://github.com/kamyu104/GoogleKickStart-2020/blob/main/Round%20D/locked_doors.py
class TreeInfos(object):  # Time: O(NlogN), Space: O(NlogN), N is the number of nodes
    def __init__(self, children):
        def preprocess(curr, parent, weight):
            if parent != -1:
                W[curr].append(weight)
                P[curr].append(parent)  # ancestors of the node i
            i = 0
            while i < len(P[curr]) and i < len(P[P[curr][i]]):
                W[curr].append(max(W[curr][i], W[P[curr][i]][i]))
                P[curr].append(P[P[curr][i]][i])
                i += 1
            C[0] += 1
            L[curr] = C[0]  # the subtree of the node i is represented by traversal index L[i]..R[i]

        def divide(curr, parent, weight):
            stk.append(partial(postprocess, curr))
            for child, w in reversed(children[curr]):
                if child == parent:
                    continue
                stk.append(partial(divide, child, curr, w))
            stk.append(partial(preprocess, curr, parent, weight))

        def postprocess(curr):
            R[curr] = C[0]  # the subtree of the node i is represented by traversal index L[i]..R[i]

        N = len(children)
        L, R, P, W, C = [0]*N, [0]*N, [[] for _ in xrange(N)], [[] for _ in xrange(N)], [-1]
        for i in xrange(N):
            if L[i]:
                continue
            stk = []
            stk.append(partial(divide, i, -1, 0))
            while stk:
                stk.pop()()
        self.L, self.R, self.P, self.W = L, R, P, W
    
    def is_ancestor(self, a, b):  # includes itself
        return self.L[a] <= self.L[b] <= self.R[b] <= self.R[a]
    
    def max_weights(self, a, b):
        def binary_lift(a, b):
            w = 0
            for i in reversed(xrange(len(self.P[a]))):  # O(logN)
                if i < len(self.P[a]) and not self.is_ancestor(self.P[a][i], b):
                    w = max(w, self.W[a][i])
                    a = self.P[a][i]
            return max(w, self.W[a][0])

        w = 0
        if not self.is_ancestor(a, b):
            w = max(w, binary_lift(a, b))
        if not self.is_ancestor(b, a):
            w = max(w, binary_lift(b, a))
        return w

    
class UnionFind(object):  # Time: O(n * Î±(n)), Space: O(n)
    def __init__(self, n):
        self.set = range(n)
        self.rank = [0]*n

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y):
        x_root, y_root = map(self.find_set, (x, y))
        if x_root == y_root:
            return False
        if self.rank[x_root] < self.rank[y_root]:  # union by rank
            self.set[x_root] = y_root
        elif self.rank[x_root] > self.rank[y_root]:
            self.set[y_root] = x_root
        else:
            self.set[y_root] = x_root
            self.rank[x_root] += 1
        return True


class DistanceLimitedPathsExist(object):

    def __init__(self, n, edgeList):
        """
        :type n: int
        :type edgeList: List[List[int]]
        """
        edgeList.sort(key = lambda x:x[2])
        self.__uf = UnionFind(n)
        self.__adj = [[] for _ in xrange(n)]
        for index, (i, j, weight) in enumerate(edgeList):
            if not self.__uf.union_set(i, j):
                continue
            self.__adj[i].append((j, weight))
            self.__adj[j].append((i, weight))
        self.__tree_infos = TreeInfos(self.__adj)

    def query(self, p, q, limit):
        """
        :type p: int
        :type q: int
        :type limit: int
        :rtype: bool
        """
        if self.__uf.find_set(p) != self.__uf.find_set(q):
            return False
        return self.__tree_infos.max_weights(p, q) < limit


# Time:  ctor:  O(mlogm + m * Î±(n) * logm) ~= O(mlogm)
#        query: O(logm + Î±(n) * logm) ~= O(logm)
# Space: O(n + m * Î±(n) + m) ~= O(n + m)
import collections
import sortedcontainers
import bisect


class SnapshotArray(object):

    def __init__(self, length):
        """
        :type length: int
        """
        self.__snaps = collections.defaultdict(lambda:sortedcontainers.SortedList([(0, 0)]))

    def set(self, index, val, snap_id):
        """
        :type index: int
        :type val: int
        :rtype: None
        """
        i = self.__snaps[index].bisect_left((snap_id, float("-inf")))
        if i != len(self.__snaps[index]) and self.__snaps[index][i][0] == snap_id:
            self.__snaps[index].remove(self.__snaps[index][i])
        self.__snaps[index].add((snap_id, val))

    def get(self, index, snap_id):
        """
        :type index: int
        :type snap_id: int
        :rtype: int
        """
        i = self.__snaps[index].bisect_left((snap_id+1, float("-inf"))) - 1
        return self.__snaps[index][i][1]   
 

class VersionedUnionFind(object):  # Time: O(n * Î±(n)), Space: O(n)

    def __init__(self, n):
        self.snap_id = 0
        self.set = SnapshotArray(n)
        for i in xrange(n):
            self.set.set(i, i, self.snap_id)
        self.rank = SnapshotArray(n)

    def find_set(self, x, snap_id):
        stk = []
        while self.set.get(x, snap_id) != x:  # path compression
            stk.append(x)
            x = self.set.get(x, snap_id)
        while stk:
            self.set.set(stk.pop(), x, snap_id)
        return x

    def union_set(self, x, y):
        x_root = self.find_set(x, self.snap_id)
        y_root = self.find_set(y, self.snap_id)
        if x_root == y_root:
            return False
        if self.rank.get(x_root, self.snap_id) < self.rank.get(y_root, self.snap_id):  # union by rank
            self.set.set(x_root, y_root, self.snap_id)
        elif self.rank.get(x_root, self.snap_id) > self.rank.get(y_root, self.snap_id):
            self.set.set(y_root, x_root, self.snap_id)
        else:
            self.set.set(y_root, x_root, self.snap_id)
            self.rank.set(x_root, self.rank.get(x_root, self.snap_id)+1, self.snap_id)
        return True

    def snap(self):
        self.snap_id += 1


class DistanceLimitedPathsExist2(object):

    def __init__(self, n, edgeList):
        """
        :type n: int
        :type edgeList: List[List[int]]
        """
        edgeList.sort(key = lambda x:x[2])
        self.__uf = VersionedUnionFind(n)
        self.__weights = []
        for index, (i, j, weight) in enumerate(edgeList):
            if not self.__uf.union_set(i, j):
                continue
            self.__uf.snap()
            self.__weights.append(weight)  

    def query(self, p, q, limit):
        """
        :type p: int
        :type q: int
        :type limit: int
        :rtype: bool
        """
        snap_id = bisect.bisect_left(self.__weights, limit)-1
        if snap_id == -1:
            return False
        return self.__uf.find_set(p, snap_id) == self.__uf.find_set(q, snap_id)

# 1728 Hard 1728 Cat and Mouse II

In [ ]:
# Time:  O((m * n)^2 * (m + n))
# Space: O((m * n)^2)

import collections


class Solution(object):
    def canMouseWin(self, grid, catJump, mouseJump):
        """
        :type grid: List[str]
        :type catJump: int
        :type mouseJump: int
        :rtype: bool
        """
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        DRAW, MOUSE, CAT = range(3)
        def parents(m, c, t):
            if t == CAT:
                for nm in graph[m, MOUSE^CAT^t]:
                    yield nm, c, MOUSE^CAT^t
            else:
                for nc in graph[c, MOUSE^CAT^t]:
                    yield m, nc, MOUSE^CAT^t

        R, C = len(grid), len(grid[0])
        N = R*C
        WALLS = set()
        FOOD, MOUSE_START, CAT_START = [-1]*3
        for r in xrange(R):
            for c in xrange(C):
                if grid[r][c] == 'M':
                    MOUSE_START = r*C + c
                elif grid[r][c] == 'C':
                    CAT_START = r*C + c
                elif grid[r][c] == 'F':
                    FOOD = r*C + c
                elif grid[r][c] == '#':
                    WALLS.add(r*C + c)

        graph = collections.defaultdict(set)
        jump = {MOUSE:mouseJump, CAT:catJump}
        for r in xrange(R):
            for c in xrange(C):
                if grid[r][c] == '#':
                    continue
                pos = r*C + c
                for t in [MOUSE, CAT]:
                    for dr, dc in directions:
                        for d in xrange(jump[t]+1):
                            nr, nc = r+dr*d, c+dc*d
                            if not (0 <= nr < R and 0 <= nc < C and grid[nr][nc] != '#'):
                                break
                            graph[pos, t].add(nr*C + nc)

        degree = {}
        for m in xrange(N):
            for c in xrange(N):
                degree[m, c, MOUSE] = len(graph[m, MOUSE])
                degree[m, c, CAT] = len(graph[c, CAT])
        color = collections.defaultdict(int)
        q = collections.deque()
        for i in xrange(N):
            if i in WALLS or i == FOOD:
                continue
            color[FOOD, i, CAT] = MOUSE
            q.append((FOOD, i, CAT, MOUSE))
            color[i, FOOD, MOUSE] = CAT
            q.append((i, FOOD, MOUSE, CAT))
            for t in [MOUSE, CAT]:
                color[i, i, t] = CAT
                q.append((i, i, t, CAT))
        while q:
            i, j, t, c = q.popleft()
            for ni, nj, nt in parents(i, j, t):
                if color[ni, nj, nt] != DRAW:
                    continue
                if nt == c:
                    color[ni, nj, nt] = c
                    q.append((ni, nj, nt, c))
                    continue
                degree[ni, nj, nt] -= 1
                if not degree[ni, nj, nt]:
                    color[ni, nj, nt] = c
                    q.append((ni, nj, nt, c))
        return color[MOUSE_START, CAT_START, MOUSE] == MOUSE


# Time:  O((m * n)^2 * (m + n))
# Space: O((m * n)^2)
import collections


class Solution2(object):
    def canMouseWin(self, grid, catJump, mouseJump):
        """
        :type grid: List[str]
        :type catJump: int
        :type mouseJump: int
        :rtype: bool
        """
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        DRAW, MOUSE, CAT = range(3)
        def parents(m, c, t):
            if t == CAT:
                for nm in graph[m, MOUSE^CAT^t]:
                    yield nm, c, MOUSE^CAT^t
            else:
                for nc in graph[c, MOUSE^CAT^t]:
                    yield m, nc, MOUSE^CAT^t

        R, C = len(grid), len(grid[0])
        N = R*C
        WALLS = set()
        FOOD, MOUSE_START, CAT_START = [-1]*3
        for r in xrange(R):
            for c in xrange(C):
                if grid[r][c] == 'M':
                    MOUSE_START = r*C + c
                elif grid[r][c] == 'C':
                    CAT_START = r*C + c
                elif grid[r][c] == 'F':
                    FOOD = r*C + c
                elif grid[r][c] == '#':
                    WALLS.add(r*C + c)
        graph = collections.defaultdict(set)
        jump = {MOUSE:mouseJump, CAT:catJump}
        for r in xrange(R):
            for c in xrange(C):
                if grid[r][c] == '#':
                    continue
                pos = r*C + c
                for t in [MOUSE, CAT]:
                    for dr, dc in directions:
                        for d in xrange(jump[t]+1):
                            nr, nc = r+dr*d, c+dc*d
                            if not (0 <= nr < R and 0 <= nc < C and grid[nr][nc] != '#'):
                                break
                            graph[pos, t].add(nr*C + nc)

        degree = {}
        for m in xrange(N):
            for c in xrange(N):
                # degree[m, c, MOUSE] = len(graph[m, MOUSE])
                degree[m, c, CAT] = len(graph[c, CAT])
        color = collections.defaultdict(int)
        q1 = collections.deque()
        # q2 = collections.deque()
        for i in xrange(N):
            if i in WALLS or i == FOOD:
                continue
            color[FOOD, i, CAT] = MOUSE
            q1.append((FOOD, i, CAT))
            color[i, FOOD, MOUSE] = CAT
            # q2.append((i, FOOD, MOUSE))
            for t in [MOUSE, CAT]:
                color[i, i, t] = CAT
                # q2.append((i, i, t))
        while q1:
            i, j, t = q1.popleft()
            for ni, nj, nt in parents(i, j, t):
                if color[ni, nj, nt] != DRAW:
                    continue
                if t == CAT:
                    color[ni, nj, nt] = MOUSE
                    q1.append((ni, nj, nt))
                    continue
                degree[ni, nj, nt] -= 1
                if not degree[ni, nj, nt]:
                    color[ni, nj, nt] = MOUSE
                    q1.append((ni, nj, nt))
        # while q2:
        #     i, j, t = q2.popleft()
        #     for ni, nj, nt in parents(i, j, t):
        #         if color[ni, nj, nt] != DRAW:
        #             continue
        #         if t == MOUSE:
        #             color[ni, nj, nt] = CAT
        #             q2.append((ni, nj, nt))
        #             continue
        #         degree[ni, nj, nt] -= 1
        #         if not degree[ni, nj, nt]:
        #             color[ni, nj, nt] = CAT
        #             q2.append((ni, nj, nt))
        return color[MOUSE_START, CAT_START, MOUSE] == MOUSE

# 1735 Hard 1735 Count Ways to Make Array With Product

In [ ]:
# Time:  O(sqrt(m) + n + q * (logm + sqrt(m)/log(sqrt(m)))), m is max(k for _, k in queries)
# Space: O(sqrt(m) + n + logm)

import collections


class Solution(object):
    def waysToFillArray(self, queries):
        """
        :type queries: List[List[int]]
        :rtype: List[int]
        """
        MOD = 10**9+7
        fact, inv, inv_fact = [[1]*2 for _ in xrange(3)]
        def nCr(n, k):
            while len(inv) <= n:  # lazy initialization
                fact.append(fact[-1]*len(inv) % MOD)
                inv.append(inv[MOD%len(inv)]*(MOD-MOD//len(inv)) % MOD)  # https://cp-algorithms.com/algebra/module-inverse.html
                inv_fact.append(inv_fact[-1]*inv[-1] % MOD)
            return (fact[n]*inv_fact[n-k] % MOD) * inv_fact[k] % MOD

        def linear_sieve_of_eratosthenes(n):  # Time: O(n), Space: O(n)
            primes = []
            spf = [-1]*(n+1)  # the smallest prime factor
            for i in xrange(2, n+1):
                if spf[i] == -1:
                    spf[i] = i
                    primes.append(i)
                for p in primes:
                    if i*p > n or p > spf[i]:
                        break
                    spf[i*p] = p
            return primes

        def prime_factors(x):
            factors = collections.Counter()
            for p in primes:
                if x < p:
                    break
                while x%p == 0:
                    factors[p] += 1
                    x //= p
            if x != 1:
                factors[x] += 1
            return factors

        primes = linear_sieve_of_eratosthenes(int(max(k for _, k in queries)**0.5))
        result = []
        for n, k in queries:
            total = 1
            for c in prime_factors(k).itervalues():
                total *= nCr(n+c-1, c)  # H(n, c) = nCr(n+c-1, n)
            result.append(total % MOD)
        return result

# 1739 Hard 1739 Building Boxes

In [ ]:
# Time:  O(1)
# Space: O(1)

import math


class Solution(object):
    def minimumBoxes(self, n):
        """
        :type n: int
        :rtype: int
        """
        # find max h s.t. sum(k*(k+1)//2 for k in xrange(1, h+1)) <= n
        # => find max h s.t. h*(h+1)*(h+2)//6 <= n
        h = int((6*n)**(1.0/3))  
        if h*(h+1)*(h+2) > 6*n:
            # (h-1)*h*(h+1) < h^3 <= 6n < h*(h+1)*(h+2) < (h+1)^3
            h -= 1
        n -= h*(h+1)*(h+2)//6
        d = int(math.ceil((-1+(1+8*n)**0.5)/2))  # find min d s.t. d*(d+1)//2 >= n
        return h*(h+1)//2 + d

# 1745 Hard 1745 Palindrome Partitioning IV

In [ ]:
# Time:  O(n^2)
# Space: O(n)

class Solution(object):
    def checkPartitioning(self, s):
        """
        :type s: str
        :rtype: bool
        """
        def manacher(s):
            s = '^#' + '#'.join(s) + '#$'
            P = [0]*len(s)
            C, R = 0, 0
            for i in xrange(1, len(s)-1):
                i_mirror = 2*C-i
                if R > i:
                    P[i] = min(R-i, P[i_mirror])
                while s[i+1+P[i]] == s[i-1-P[i]]:
                    P[i] += 1
                if i+P[i] > R:
                    C, R = i, i+P[i]
            return P

        P = manacher(s)
        prefix, suffix = [], []
        for i in xrange(2, len(P)-2):
            if i-1-P[i] == 0:
                prefix.append(i)
            if i+1+P[i] == len(P)-1:
                suffix.append(i)
        for i in prefix:
            for j in suffix:
                left, right = i+1+P[i], j-1-P[j]
                if left > right:
                    continue
                mid = left + (right-left)//2
                if P[mid] >= mid-left:
                    return True
        return False


# Time:  O(n^2)
# Space: O(n^2)
class Solution2(object):
    def checkPartitioning(self, s):
        """
        :type s: str
        :rtype: bool
        """        
        dp = [[False]*len(s) for _ in xrange(len(s))]
        for i in reversed(xrange(len(s))):
            for j in xrange(i, len(s)):
                if s[i] == s[j] and (j-i < 2 or dp[i+1][j-1]):
                    dp[i][j] = True
        for i in xrange(1, len(s)-1):
            if not dp[0][i-1]:
                continue
            for j in xrange(i+1, len(s)):
                if not dp[j][-1]:
                    continue
                if dp[i][j-1]:
                    return True
        return False

# 1751 Hard 1751 Maximum Number of Events That Can Be Attended II

In [ ]:
# Time:  O(nlogn + n * k)
# Space: O(n * k)

import bisect


class Solution(object):
    def maxValue(self, events, k):
        """
        :type events: List[List[int]]
        :type k: int
        :rtype: int
        """
        events.sort(key=lambda x: x[1])
        sorted_ends = [x[1] for x in events]
        dp = [[0]*(k+1) for _ in xrange(len(events)+1)]
        for i in xrange(1, len(events)+1):
            prev_i_m_1 = bisect.bisect_left(sorted_ends, events[i-1][0])-1
            for j in xrange(1, k+1):
                dp[i][j] = max(dp[i-1][j], dp[prev_i_m_1+1][j-1]+events[i-1][2])
        return dp[-1][-1]


# Time:  O(nlogn + n * k)
# Space: O(n * k)
import bisect


class Solution2(object):
    def maxValue(self, events, k):
        """
        :type events: List[List[int]]
        :type k: int
        :rtype: int
        """
        events.sort()
        sorted_starts = [x[0] for x in events]
        dp = [[0]*(k+1) for _ in xrange(len(events)+1)]
        for i in reversed(xrange(len(events))):
            next_i = bisect.bisect_right(sorted_starts, events[i][1])-1
            for j in xrange(1, k+1):
                dp[i][j] = max(dp[i+1][j], dp[next_i+1][j-1]+events[i][2])
        return dp[0][-1]

# 1755 Hard 1755 Closest Subsequence Sum

In [ ]:
# Time:  O(n * 2^(n/2))
# Space: O(2^(n/2))

import bisect


class Solution(object):
    def minAbsDifference(self, nums, goal):
        """
        :type nums: List[int]
        :type goal: int
        :rtype: int
        """
        mx, mn = sum(x for x in nums if x > 0), sum(x for x in nums if x < 0)
        if goal > mx:
            return goal-mx
        if goal < mn:
            return mn-goal
        result = abs(goal)
        sums1 = set([0])
        for i in xrange(len(nums)//2):
            for x in list(sums1):
                if x+nums[i] in sums1:
                    continue
                sums1.add(x+nums[i])
                result = min(result, abs(goal-x-nums[i]))  # case of right half part is 0
        sorted_sums1 = sorted(sums1)  # Time: O((n/2) * 2^(n/2)) = O(n * 2^(n/2)), Space: O(2^(n/2))
        sums2 = set([0])
        for i in xrange(len(nums)//2, len(nums)):
            for x in list(sums2):
                if x+nums[i] in sums2:
                    continue
                sums2.add(x+nums[i])
                ni = bisect.bisect_left(sorted_sums1, goal-x-nums[i])  # Time: O(2^(n/2)) * O(n/2)
                if ni < len(sorted_sums1):
                    result = min(result, abs(goal-x-nums[i]-sorted_sums1[ni]))
                if ni > 0:
                    result = min(result, abs(goal-x-nums[i]-sorted_sums1[ni-1]))
                if result == 0:
                    return result
        return result

# 1761 Hard 1761 Minimum Degree of a Connected Trio in a Graph

In [ ]:
# Time:  O(n^3)
# Space: O(n^2)

class Solution(object):
    def minTrioDegree(self, n, edges):
        """
        :type n: int
        :type edges: List[List[int]]
        :rtype: int
        """
        adj = [set() for _ in xrange(n+1)]
        degree = [0]*(n+1)
        for u, v in edges:
            adj[min(u, v)].add(max(u, v))
            degree[u] += 1
            degree[v] += 1
        result = float("inf")
        for u in xrange(1, n+1):
            for v in adj[u]:
                for w in adj[u]:
                    if v < w and w in adj[v]:
                        result = min(result, degree[u]+degree[v]+degree[w] - 6)
        return result if result != float("inf") else -1

# 1766 Hard 1766 Tree of Coprimes

In [ ]:
# Time:  O(50 * n) = O(n)
# Space: O(n)

import collections
import fractions


class Solution(object):
    def getCoprimes(self, nums, edges):
        """
        :type nums: List[int]
        :type edges: List[List[int]]
        :rtype: List[int]
        """        
        def iter_dfs(nums, adj):
            result = [-1]*len(nums)
            path = collections.defaultdict(list)
            stk = [(1, (-1, 0, 0))]
            while stk:
                step, params = stk.pop()
                if step == 1:
                    prev, node, depth = params
                    stk.append((4, (node,)))
                    stk.append((3, (prev, node, depth)))
                    stk.append((2, (node,)))
                elif step == 2:
                    node = params[0]
                    max_d = -1
                    for x in path.iterkeys():
                        if fractions.gcd(nums[node], x) != 1:
                            continue
                        if path[x][-1][1] > max_d:
                            max_d = path[x][-1][1]
                            result[node] = path[x][-1][0]
                elif step == 3:
                    prev, node, depth = params
                    path[nums[node]].append((node, depth))
                    for nei in adj[node]:
                        if nei == prev:
                            continue
                        stk.append((1, (node, nei, depth+1)))
                elif step == 4:
                    node = params[0]
                    path[nums[node]].pop()
                    if not path[nums[node]]:
                        path.pop(nums[node])
            return result

        adj = collections.defaultdict(list)
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        return iter_dfs(nums, adj)


# Time:  O(50 * n) = O(n)
# Space: O(n)
import collections
import fractions


class Solution2(object):
    def getCoprimes(self, nums, edges):
        """
        :type nums: List[int]
        :type edges: List[List[int]]
        :rtype: List[int]
        """        
        def dfs(nums, adj, prev, node, depth, path, result):
            max_d = -1
            for x in path.iterkeys():
                if fractions.gcd(nums[node], x) != 1:
                    continue
                if path[x][-1][1] > max_d:
                    max_d = path[x][-1][1]
                    result[node] = path[x][-1][0]
            path[nums[node]].append((node, depth))
            for nei in adj[node]:
                if nei == prev:
                    continue
                dfs(nums, adj, node, nei, depth+1, path, result)
            path[nums[node]].pop()
            if not path[nums[node]]:
                path.pop(nums[node])

        adj = collections.defaultdict(list)
        for u, v in edges:
            adj[u].append(v)
            adj[v].append(u)
        result = [-1]*len(nums)
        path = collections.defaultdict(list)
        dfs(nums, adj, -1, 0, 0, path, result)
        return result

# 1770 Hard 1770 Maximum Score from Performing Multiplication Operations

In [ ]:
# Time:  O(m^2)
# Space: O(m)

class Solution(object):
    def maximumScore(self, nums, multipliers):
        """
        :type nums: List[int]
        :type multipliers: List[int]
        :rtype: int
        """
        dp = [0]*(len(multipliers)+1)
        for l, m in enumerate(reversed(multipliers), start=len(nums)-len(multipliers)):
            dp = [max(m*nums[i]+dp[i+1], m*nums[i+l]+dp[i]) for i in xrange(len(dp)-1)]
        return dp[0]

# 1771 Hard 1771 Maximize Palindrome Length From Subsequences

In [ ]:
# Time:  O((m + n)^2)
# Space: O((m + n)^2)

class Solution(object):
    def longestPalindrome(self, word1, word2):
        """
        :type word1: str
        :type word2: str
        :rtype: int
        """
        s = word1+word2
        dp = [[0]*len(s) for _ in xrange(len(s))]
        result = 0
        for j in xrange(len(s)):
            dp[j][j] = 1
            for i in reversed(xrange(j)):
                if s[i] == s[j]:
                    dp[i][j] = 2 if i+1 == j else dp[i+1][j-1] + 2
                    if i < len(word1) <= j:
                        result = max(result, dp[i][j])
                else:
                    dp[i][j] = max(dp[i+1][j], dp[i][j-1])
        return result


# Time:  O((m + n)^2)
# Space: O((m + n)^2)
class Solution2(object):
    def longestPalindrome(self, word1, word2):
        """
        :type word1: str
        :type word2: str
        :rtype: int
        """
        s = word1+word2
        dp = [[0]*len(s) for _ in xrange(len(s))]
        for j in xrange(len(s)):
            dp[j][j] = 1
            for i in reversed(xrange(j)):
                if s[i] == s[j]:
                    dp[i][j] = 2 if i+1 == j else dp[i+1][j-1] + 2
                else:
                    dp[i][j] = max(dp[i+1][j], dp[i][j-1])
        return max([dp[i][j] for i in xrange(len(word1)) for j in xrange(len(word1), len(s)) if s[i] == s[j]] or [0])

# 1776 Hard 1776 Car Fleet II

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def getCollisionTimes(self, cars):
        """
        :type cars: List[List[int]]
        :rtype: List[float]
        """
        stk = []
        result = [-1.0]*len(cars)
        for i in reversed(xrange(len(cars))):
            p, s = cars[i]
            while stk and (cars[stk[-1]][1] >= s or 
                           0 < result[stk[-1]] <= float(cars[stk[-1]][0]-p)/(s-cars[stk[-1]][1])):
                stk.pop()
            if stk:
                result[i] = float(cars[stk[-1]][0]-p)/(s-cars[stk[-1]][1])
            stk.append(i)
        return result

# 1782 Hard 1782 Count Pairs Of Nodes

In [ ]:
# Time:  O(n + e + q)
# Space: O(n + e)

import collections
import itertools


# pure counting solution
class Solution(object):
    def countPairs(self, n, edges, queries):
        """
        :type n: int
        :type edges: List[List[int]]
        :type queries: List[int]
        :rtype: List[int]
        """
        degree = [0]*(n+1)
        shared = collections.Counter((min(n1, n2), max(n1, n2)) for n1, n2 in edges)
        for u, v in edges:
            degree[u] += 1
            degree[v] += 1
        cnt = [0]*(2*(max(degree[1:])+1))
        count = collections.Counter(degree[1:])
        for i, j in itertools.product(count, count):  # Time: O(d^2) = O(e)
            if i < j:
                cnt[i+j] += count[i]*count[j]
            elif i == j:
                cnt[i+j] += count[i]*(count[i]-1)//2
        for (i, j), shared_degree in shared.iteritems():
            cnt[degree[i]+degree[j]] -= 1
            cnt[degree[i]+degree[j]-shared_degree] += 1
        for i in reversed(xrange(len(cnt)-1)):  # accumulate
            cnt[i] += cnt[i+1]
        return [cnt[q+1] if q+1 < len(cnt) else 0 for q in queries]


# Time:  O(nlogn + q * (n + e))
# Space: O(n + e)
import collections


# two pointers solution
class Solution2(object):
    def countPairs(self, n, edges, queries):
        """
        :type n: int
        :type edges: List[List[int]]
        :type queries: List[int]
        :rtype: List[int]
        """
        degree = [0]*(n+1)
        shared = collections.Counter((min(n1, n2), max(n1, n2)) for n1, n2 in edges)
        for n1, n2 in edges:
            degree[n1] += 1
            degree[n2] += 1
        sorted_degree = sorted(degree)
        result = []
        for k, q in enumerate(queries):
            left, right = 1, n
            cnt = 0
            while left < right:
                if q < sorted_degree[left]+sorted_degree[right]:
                    cnt += right-left
                    right -= 1
                else:
                    left += 1
            for (i, j), shared_degree in shared.iteritems():
                if degree[i]+degree[j]-shared_degree <= q < degree[i]+degree[j]:
                    cnt -= 1
            result.append(cnt)
        return result

# 1787 Hard 1787 Make the XOR of All Segments Equal to Zero

In [ ]:
# Time:  O(n + k * m), m is the max number of nums
# Space: O(min(k * m, n))

import collections


class Solution(object):
    def minChanges(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        def one_are_not_from_nums(nums, cnts):
            mxs = [cnts[i].most_common(1)[0][1] for i in xrange(k)]
            return len(nums) - (sum(mxs)-min(mxs))

        def all_are_from_nums(nums, cnts):
            dp = {0:0}
            for cnt in cnts:
                new_dp = collections.defaultdict(int)
                for x in dp.iterkeys():
                    for y in cnt.iterkeys():
                        new_dp[x^y] = max(new_dp[x^y], dp[x]+cnt[y])
                dp = new_dp
            return len(nums)-dp[0]
          
        cnts = [collections.Counter(nums[j] for j in xrange(i, len(nums), k)) for i in xrange(k)]
        return min(one_are_not_from_nums(nums, cnts), all_are_from_nums(nums, cnts))

# 1788 Hard 1788 Maximize the Beauty of the Garden

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def maximumBeauty(self, flowers):
        """
        :type flowers: List[int]
        :rtype: int
        """
        lookup = {}
        prefix = [0]
        result = float("-inf")
        for i, f in enumerate(flowers):
            prefix.append(prefix[-1]+f if f > 0 else prefix[-1])
            if not f in lookup:
                lookup[f] = i
                continue
            result = max(result, 2*f+prefix[i+1]-prefix[lookup[f]] if f < 0 else prefix[i+1]-prefix[lookup[f]])
        return result

# 1793 Hard 1793 Maximum Score of a Good Subarray

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def maximumScore(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        result = curr = nums[k]
        left = right = k
        while left-1 >= 0 or right+1 < len(nums):
            # choosing larger one to expand is always better than or equal to choosing smaller one
            if (nums[left-1] if left-1 >= 0 else 0) <= (nums[right+1] if right+1 < len(nums) else 0):
                right += 1
            else:
                left -= 1
            curr = min(curr, nums[left], nums[right])
            result = max(result, curr*(right-left+1))
        return result


# Time:  O(nlogn)
# Space: O(n)
import bisect


class Solution2(object):
    def maximumScore(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        def score(nums, k):
            prefix = [nums[k]]*(k+1)
            for i in reversed(xrange(k)):
                prefix[i] = min(prefix[i+1], nums[i])
            result = right = nums[k]
            for j in xrange(k+1, len(nums)):
                right = min(right, nums[j])
                i = bisect.bisect_left(prefix, right)
                if i >= 0:
                    result = max(result, right*(j-i+1))
            return result

        return max(score(nums, k), score(nums[::-1], len(nums)-1-k))
 

# 1799 Hard 1799 Maximize Score After N Operations

In [ ]:
# Time:  O(n^2 * 2^n)
# Space: O(2^n)

import itertools
from fractions import gcd


class Solution(object):
    def maxScore(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        def popcount(n):
            count = 0
            while n:
                n &= n-1
                count += 1
            return count

        def bits(mask):
            result = []
            i = 0
            while mask:
                if mask&1:
                    result.append(i)
                i += 1
                mask >>= 1
            return result
            
        dp = [0]*(2**len(nums))
        for mask in xrange(3, len(dp)):
            cnt = popcount(mask)
            if cnt%2:
                continue
            for i, j in itertools.combinations(bits(mask), 2):  # Time: O(n^2)
                dp[mask] = max(dp[mask], cnt//2*gcd(nums[i], nums[j]) + dp[mask^(1<<i)^(1<<j)])
        return dp[-1]

# 1803 Hard 1803 Count Pairs With XOR in a Range

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


# dp solution
class Solution(object):
    def countPairs(self, nums, low, high):
        """
        :type nums: List[int]
        :type low: int
        :type high: int
        :rtype: int
        """
        def count(nums, x):
            result = 0
            dp = collections.Counter(nums)
            while x:
                if x&1:
                    result += sum(dp[(x^1)^k]*dp[k] for k in dp.iterkeys())//2  # current limit is xxxxx1*****, count xor pair with xxxxx0***** pattern
                dp = collections.Counter({k>>1: dp[k]+dp[k^1] for k in dp.iterkeys()})
                x >>= 1
            return result
    
        return count(nums, high+1)-count(nums, low)


# Time:  O(n)
# Space: O(n)
# trie solution
class Trie(object):
    def __init__(self):
        self.__root = {}
        
    def insert(self, num):
        node = self.__root
        for i in reversed(xrange(32)):
            curr = (num>>i) & 1
            if curr not in node:
                node[curr] = {"_count":0}
            node = node[curr]
            node["_count"] += 1
                
    def query(self, num, limit):
        node, result = self.__root, 0
        for i in reversed(xrange(32)):
            curr = (num>>i) & 1
            bit = (limit>>i) & 1
            if bit:
                if curr in node:
                    result += node[0^curr]["_count"]  # current limit is xxxxx1*****, count xor pair with xxxxx0***** pattern
            if bit^curr not in node:
                break
            node = node[bit^curr]
        return result


class Solution2(object):
    def countPairs(self, nums, low, high):
        """
        :type nums: List[int]
        :type low: int
        :type high: int
        :rtype: int
        """
        result = 0
        trie = Trie()
        for x in nums:
            result += trie.query(x, high+1)-trie.query(x, low)
            trie.insert(x)
        return result

# 1808 Hard 1808 Maximize Number of Nice Divisors

In [ ]:
# Time:  O(logn)
# Space: O(1)

# variant of "343. integer break"
class Solution(object):
    def maxNiceDivisors(self, primeFactors):
        """
        :type primeFactors: int
        :rtype: int
        """
        # given a1 + a2 + ... + ak <= n, find max of a1 * a2 * ... * ak 
        # => given a1 + a2 + ... + ak = n, find max of a1 * a2 * ... * ak 
        # => ai is either 3 or 2, see proof in "343. integer break"
        MOD = 10**9 + 7
        if primeFactors <= 3:
            return primeFactors
        if primeFactors % 3 == 0:  # 6 => 3*3
            return pow(3, primeFactors//3, MOD)
        if primeFactors % 3 == 1:  # 4 => 2*2 
            return (2*2*pow(3, (primeFactors-4)//3, MOD)) % MOD
        return (2*pow(3, (primeFactors-2)//3, MOD)) % MOD  # 5 => 2*3

# 1815 Hard 1815 Maximum Number of Groups Getting Fresh Donuts

In [ ]:
# Time:  O((b/2) * (n/(b/2)+1)^(b/2))
# Space: O((n/(b/2)+1)^(b/2))

# greedy + memoization solution
class Solution(object):
    def maxHappyGroups(self, batchSize, groups):
        """
        :type batchSize: int
        :type groups: List[int]
        :rtype: int
        """
        def memoization(batchSize, count, mask, remain, lookup):
            if lookup[mask] == 0:
                a_remain = 0
                if remain in count:
                    curr, basis = mask, 1
                    for i, c in count.iteritems():
                        if i == remain:
                            break
                        basis *= c+1
                        curr //= c+1
                    a_remain = curr%(count[remain]+1)
                result = 0
                if a_remain:
                    result = max(result, int(remain == 0) + memoization(batchSize, count, mask-basis, 0, lookup))
                else:
                    curr, basis = mask, 1
                    for i, c in count.iteritems():
                        if curr%(c+1):
                            result = max(result, int(remain == 0) + memoization(batchSize, count, mask-basis, (remain-i)%batchSize, lookup))
                        basis *= c+1
                        curr //= c+1
                lookup[mask] = result
            return lookup[mask]
    
        count = [0]*batchSize
        for i in groups:
            count[i%len(count)] += 1
        result = count[0]
        count[0] = 0
        for i in xrange(1, len(count)//2+1):  # optimization
            pair_count = min(count[i], count[len(count)-i]) if 2*i != len(count) else count[i]//2
            result += pair_count
            count[i] -= pair_count
            count[len(count)-i] -= pair_count
        new_count = {i:c for i, c in enumerate(count) if c}
        max_mask = reduce(lambda total, c: total*(c+1), new_count.itervalues(), 1)
        lookup = [0]*max_mask
        return result+memoization(batchSize, new_count, max_mask-1, 0, lookup)


# Time:  O((b/2) * (n/(b/2)+1)^(b/2))
# Space: O((n/(b/2)+1)^(b/2))
# dp solution
class Solution2(object):
    def maxHappyGroups(self, batchSize, groups):
        """
        :type batchSize: int
        :type groups: List[int]
        :rtype: int
        """
        count = [0]*batchSize
        for i in groups:
            count[i%len(count)] += 1
        result = count[0]
        count[0] = 0
        for i in xrange(1, len(count)//2+1):  # optimization
            pair_count = min(count[i], count[len(count)-i]) if 2*i != len(count) else count[i]//2
            result += pair_count
            count[i] -= pair_count
            count[len(count)-i] -= pair_count
        new_count = {i:c for i, c in enumerate(count) if c}
        max_mask = reduce(lambda total, c: total*(c+1), new_count.itervalues(), 1)
        dp = [0]*max_mask
        for mask in xrange(len(dp)):
            remain = 0
            curr, basis = mask, 1
            for i, c in new_count.iteritems():
                ai = curr%(c+1)
                if ai:
                    dp[mask] = max(dp[mask], dp[mask-basis])
                remain = (remain+ai*i)%batchSize
                basis *= c+1
                curr //= c+1
            if mask != len(dp)-1 and remain == 0:
                dp[mask] += 1
        return result+dp[-1]

# 1819 Hard 1819 Number of Different Subsequences GCDs

In [ ]:
# Time:  O(n + m * (1 + 1/2 + 1/3 + ... + 1/m)) = O(n + mlogm), m is max of nums
# Space: O(n)

import fractions


class Solution(object):
    def countDifferentSubsequenceGCDs(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        max_num, nums_set = max(nums), set(nums)
        result = 0
        for i in xrange(1, max_num+1):
            d = 0
            for x in xrange(i, max_num+1, i):
                if x not in nums_set:
                    continue
                d = fractions.gcd(d, x)  # total time: O(log(min(d, x)) = O(logd), where d keeps the same or gets smaller
                if d == i:
                    result += 1
                    break
        return result

# 1825 Hard 1825 Finding MK Average

In [ ]:
# Time:  ctor:           O(1)
#        add_element:    O(logn)
#        calc_mkaverage: O(1)
# Space: O(m)

import collections
from sortedcontainers import SortedList


class MKAverage(object):

    def __init__(self, m, k):
        """
        :type m: int
        :type k: int
        """
        self.__m = m
        self.__k = k
        self.__dq = collections.deque()
        self.__sl = SortedList()
        self.__total = self.__first_k = self.__last_k = 0

    def addElement(self, num):
        """
        :type num: int
        :rtype: None
        """
        if len(self.__dq) == self.__m:
            self.__remove(self.__dq.popleft())
        self.__dq.append(num)
        self.__add(num)

    def calculateMKAverage(self):
        """
        :rtype: int
        """
        if len(self.__sl) < self.__m:
            return -1
        return (self.__total-self.__first_k-self.__last_k)//(self.__m-2*self.__k)

    def __add(self, num):
        self.__total += num
        idx = self.__sl.bisect_left(num)
        if idx < self.__k:
            self.__first_k += num
            if len(self.__sl) >= self.__k:
                self.__first_k -= self.__sl[self.__k-1]
        if idx > len(self.__sl)-self.__k:
            self.__last_k += num
            if len(self.__sl) >= self.__k:
                self.__last_k -= self.__sl[-self.__k]
        self.__sl.add(num)

    def __remove(self, num):
        self.__total -= num
        idx = self.__sl.index(num)
        if idx < self.__k:
            self.__first_k -= num
            self.__first_k += self.__sl[self.__k]
        elif idx > (len(self.__sl)-1)-self.__k:
            self.__last_k -= num
            self.__last_k += self.__sl[-1-self.__k]
        self.__sl.remove(num)

# 1830 Hard 1830 Minimum Number of Operations to Make String Sorted

In [ ]:
# Time:  O(26 * n) = O(n)
# Space: O(max_n) = O(max_n)

inv = [0, 1]


class Solution(object):
    def makeStringSorted(self, s):  # count of prev_permutation
        """
        :type s: str
        :rtype: int
        """
        def inverse(n, m):
            i = len(inv)
            while len(inv) <= n:  # lazy initialization
                inv.append(inv[m%i]*(m-m//i) % m)  # https://cp-algorithms.com/algebra/module-inverse.html
                i += 1
            return inv[n]
    
        MOD = 10**9+7
        count, result, comb_total = [0]*26, 0, 1
        for i in reversed(xrange(len(s))):
            num = ord(s[i])-ord('a') 
            count[num] += 1
            comb_total = (comb_total*(len(s)-i))*inverse(count[num], MOD)
            result = (result + (comb_total*sum(count[:num]))*inverse(len(s)-i, MOD)) % MOD
        return result

# 1835 Hard 1835 Find XOR Sum of All Pairs Bitwise AND

In [ ]:
# Time:  O(n)
# Space: O(1)

import operator


class Solution(object):
    def getXORSum(self, arr1, arr2):
        """
        :type arr1: List[int]
        :type arr2: List[int]
        :rtype: int
        """
        return reduce(operator.xor, arr1) & reduce(operator.xor, arr2)

# 1840 Hard 1840 Maximum Building Height

In [ ]:
# Time:  O(nlogn)
# Space: O(1)

class Solution(object):
    def maxBuilding(self, n, restrictions):
        """
        :type n: int
        :type restrictions: List[List[int]]
        :rtype: int
        """
        restrictions.extend([[1, 0], [n, n-1]])
        restrictions.sort()
        for i in reversed(xrange(len(restrictions)-1)):
            restrictions[i][1] = min(restrictions[i][1], restrictions[i+1][1]+(restrictions[i+1][0]-restrictions[i][0]))
        result = 0
        for i in xrange(1, len(restrictions)):
            restrictions[i][1] = min(restrictions[i][1], restrictions[i-1][1]+(restrictions[i][0]-restrictions[i-1][0]))
            left, h1 = restrictions[i-1]
            right, h2 = restrictions[i]
            result = max(result, max(h1, h2)+((right-left)-abs(h1-h2))//2)
        return result

# 1842 Hard 1842 Next Palindrome Using Same Digits

In [ ]:
# Time:  O(n)
# Space: O(1)

class Solution(object):
    def nextPalindrome(self, num):
        """
        :type num: str
        :rtype: str
        """
        def next_permutation(nums, begin, end):
            def reverse(nums, begin, end):
                left, right = begin, end-1
                while left < right:
                    nums[left], nums[right] = nums[right], nums[left]
                    left += 1
                    right -= 1

            k, l = begin-1, begin
            for i in reversed(xrange(begin, end-1)):
                if nums[i] < nums[i+1]:
                    k = i
                    break
            else:
                reverse(nums, begin, end)
                return False
            for i in reversed(xrange(k+1, end)):
                if nums[i] > nums[k]:
                    l = i
                    break
            nums[k], nums[l] = nums[l], nums[k]
            reverse(nums, k+1, end)
            return True
        
        nums = list(num)
        if not next_permutation(nums, 0, len(nums)//2):
            return ""
        for i in xrange(len(nums)//2):
            nums[-1-i] = nums[i]
        return "".join(nums)

# 1847 Hard 1847 Closest Room

In [ ]:
# Time:  O(nlogn + klogk + klogn)
# Space: O(n + k)

from sortedcontainers import SortedList


class Solution(object):
    def closestRoom(self, rooms, queries):
        """
        :type rooms: List[List[int]]
        :type queries: List[List[int]]
        :rtype: List[int]
        """
        def find_closest(ids, r):
            result, min_dist = -1, float("inf")
            i = ids.bisect_right(r)
            if i-1 >= 0 and abs(ids[i-1]-r) < min_dist:
                min_dist = abs(ids[i-1]-r)
                result = ids[i-1]
            if i < len(ids) and abs(ids[i]-r) < min_dist:
                min_dist = abs(ids[i]-r)
                result = ids[i]
            return result

        rooms.sort(key=lambda x: x[1], reverse=True)
        for i, q in enumerate(queries):
            q.append(i)
        queries.sort(key=lambda x: x[1], reverse=True)
        ids = SortedList()
        i = 0
        result = [-1]*len(queries)
        for r, s, idx in queries:
            while i < len(rooms) and rooms[i][1] >= s:
                ids.add(rooms[i][0])
                i += 1
            result[idx] = find_closest(ids, r)
        return result

    
# Time:  O(nlogn + klogk + klogn)
# Space: O(n + k)
from sortedcontainers import SortedList


class Solution2(object):
    def closestRoom(self, rooms, queries):
        """
        :type rooms: List[List[int]]
        :type queries: List[List[int]]
        :rtype: List[int]
        """
        def find_closest(ids, r):
            result, min_dist = -1, float("inf")
            i = ids.bisect_right(r)
            if i-1 >= 0 and abs(ids[i-1]-r) < min_dist:
                min_dist = abs(ids[i-1]-r)
                result = ids[i-1]
            if i < len(ids) and abs(ids[i]-r) < min_dist:
                min_dist = abs(ids[i]-r)
                result = ids[i]
            return result

        rooms.sort(key=lambda x: x[1])
        for i, q in enumerate(queries):
            q.append(i)
        queries.sort(key=lambda x: x[1])
        ids = SortedList(i for i, _ in rooms)        
        i = 0
        result = [-1]*len(queries)
        for r, s, idx in queries:
            while i < len(rooms) and rooms[i][1] < s:
                ids.remove(rooms[i][0])
                i += 1
            result[idx] = find_closest(ids, r)
        return result

# 1851 Hard 1851 Minimum Interval to Include Each Query

In [ ]:
class Solution:
    def minInterval(self, intervals: List[List[int]], queries: List[int]) -> List[int]:
        ans = [-1]*len(queries)
        queries = sorted([(query, i) for i, query in enumerate(queries)])
        intervals.sort()
        h = []
        
        itervalIndex = 0
        for query, queryIndex in queries:
            #push all intervals that include 'query' to the min heap
            while itervalIndex<len(intervals) and intervals[itervalIndex][0]<=query:
                left = intervals[itervalIndex][0]
                right = intervals[itervalIndex][1]
                size = right-left+1
                heapq.heappush(h, (size, right))
                itervalIndex += 1
            
            #pop all intervals that not includes 'query' out of the min heap
            while h and h[0][1]<query:
                heapq.heappop(h)
                
            if h: ans[queryIndex] = h[0][0]
        return ans

# 1857 Hard 1857 Largest Color Value in a Directed Graph

In [ ]:
# Time:  O(n + m)
# Space: O(n + m)

class Solution(object):
    def largestPathValue(self, colors, edges):
        """
        :type colors: str
        :type edges: List[List[int]]
        :rtype: int
        """
        adj = [[] for _ in xrange(len(colors))]
        in_degree = [0]*len(colors)
        for u, v in edges:
            adj[u].append(v)
            in_degree[v] += 1
        q = []
        for u in xrange(len(colors)):
            if not in_degree[u]:
                q.append(u)
        dp = [[0]*26 for _ in xrange(len(colors))]
        result, cnt = -1, 0
        while q:
            new_q = []
            for u in q:
                cnt += 1
                dp[u][ord(colors[u])-ord('a')] += 1
                result = max(result, dp[u][ord(colors[u])-ord('a')])
                for v in adj[u]:
                    for c in xrange(26):
                        dp[v][c] = max(dp[v][c], dp[u][c])
                    in_degree[v] -= 1
                    if not in_degree[v]:
                        new_q.append(v)
            q = new_q
        return result if cnt == len(colors) else -1

# 1862 Hard 1862 Sum of Floored Pairs

In [ ]:
# Time:  O(n/1+n/2+...+n/n) = O(nlogn), n is the max of nums
# Space: O(n)

import collections


class Solution(object):
    def sumOfFlooredPairs(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        MOD = 10**9+7
        prefix, counter = [0]*(max(nums)+1), collections.Counter(nums)
        for num, cnt in counter.iteritems():
            for j in xrange(num, len(prefix), num):
                prefix[j] += counter[num]
        for i in xrange(len(prefix)-1):
            prefix[i+1] += prefix[i]
        return reduce(lambda total, num: (total+prefix[num])%MOD, nums, 0)

# 1866 Hard 1866 Number of Ways to Rearrange Sticks With K Sticks Visible

In [ ]:
# Time:  O(n * k)
# Space: O(k)

class Solution(object):
    def rearrangeSticks(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: int
        """
        MOD = 10**9+7
        dp = [[0 for _ in xrange(k+1)] for _ in xrange(2)]
        dp[1][1] = 1
        for i in xrange(2, n+1):
            for j in xrange(1, min(i, k)+1):
                # choose the tallest as the last one which would be visible:    dp[i-1][j-1]
                # choose the non-tallest as the last one which would be hidden: (i-1)*dp[i-1][j]
                dp[i%2][j] = (dp[(i-1)%2][j-1]+(i-1)*dp[(i-1)%2][j]) % MOD 
        return dp[n%2][k]

# 1872 Hard 1872 Stone Game VIII

In [ ]:
# Time:  O(n)
# Space: O(1)

class Solution(object):
    def stoneGameVIII(self, stones):
        """
        :type stones: List[int]
        :rtype: int
        """
        for i in xrange(len(stones)-1):
            stones[i+1] += stones[i]
        return reduce(lambda curr, i: max(curr, stones[i]-curr), reversed(xrange(1, len(stones)-1)), stones[-1])

# 1879 Hard 1879 Minimum XOR Sum of Two Arrays

In [ ]:
"""
state[i] := nums1[i] has been matched.
"""
class Solution(object):
    def minimumXORSum(self, nums1, nums2):
        N = len(nums1)
        
        pq = [(0, '0'*N)]
        visited = set()
        while pq:
            s, state = heapq.heappop(pq)
            if state in visited: continue
            visited.add(state)
            
            j = state.count('1')
            if j==N: return s
            
            for i in xrange(N):
                if state[i]=='1': continue
                nextState = state[:i]+'1'+state[i+1:]
                if nextState in visited: continue
                heapq.heappush(pq, ((s+(nums1[i]^nums2[j-1]), nextState)))
        
        return float('inf')

# 1883 Hard 1883 Minimum Skips to Arrive at Meeting On Time

In [ ]:
# Time:  O(n^2)
# Space: O(n)

class Solution(object):
    def minSkips(self, dist, speed, hoursBefore):
        """
        :type dist: List[int]
        :type speed: int
        :type hoursBefore: int
        :rtype: int
        """
        def ceil(a, b):
            return (a+b-1)//b

        dp = [0]*((len(dist)-1)+1)  # dp[i]: (min time by i skips) * speed
        for i, d in enumerate(dist):
            for j in reversed(xrange(len(dp))):
                dp[j] = ceil(dp[j]+d, speed)*speed if i < len(dist)-1 else dp[j]+d
                if j-1 >= 0:
                    dp[j] = min(dp[j], dp[j-1]+d)
        target = hoursBefore*speed
        for i in xrange(len(dist)):
            if dp[i] <= target:
                return i
        return -1

# 1889 Hard 1889 Minimum Space Wasted From Packaging

In [ ]:
# Time:  O(mlogm + nlogn + mlogn)
# Space: O(1)

import bisect


class Solution(object):
    def minWastedSpace(self, packages, boxes):
        """
        :type packages: List[int]
        :type boxes: List[List[int]]
        :rtype: int
        """
        MOD = 10**9+7
        INF = float("inf")

        packages.sort()
        result = INF
        for box in boxes:
            box.sort()
            if box[-1] < packages[-1]:
                continue
            curr = left = 0
            for b in box:
                right = bisect.bisect_right(packages, b, left)
                curr += b * (right-left)
                left = right
            result = min(result, curr)
        return (result-sum(packages))%MOD if result != INF else -1

# 1896 Hard 1896 Minimum Cost to Change the Final Value of Expression

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def minOperationsToFlip(self, expression):
        """
        :type expression: str
        :rtype: int
        """
        def compute(operands, operators):
            right, left = operands.pop(), operands.pop()
            operands.append(ops[operators.pop()](left, right))

        ops = {'&':lambda x, y: [min(x[0], y[0]), min(x[1]+y[1], min(x[1], y[1])+1)],
               '|':lambda x, y: [min(x[0]+y[0], min(x[0], y[0])+1), min(x[1], y[1])]}
        precedence = {'&':0, '|':0}
        operands, operators = [], []
        for c in expression:
            if c.isdigit():
                operands.append([int(c != '0'), int(c != '1')])
            elif c == '(':
                operators.append(c)
            elif c == ')':
                while operators[-1] != '(':
                    compute(operands, operators)
                operators.pop()
            elif c in precedence:
                while operators and operators[-1] in precedence and \
                      precedence[operators[-1]] >= precedence[c]:
                    compute(operands, operators)
                operators.append(c)
        while operators:
            compute(operands, operators)
        return max(operands[-1])


# Time:  O(n)
# Space: O(n)
class Solution2(object):
    def minOperationsToFlip(self, expression):
        """
        :type expression: str
        :rtype: int
        """
        stk = [[None]*3]
        for c in expression:                                
            if c == '(':                                            
                stk.append([None]*3)
            elif c in {')', '0', '1'}:
                if c == ')':
                    dp0, dp1, _ = stk.pop()
                else:
                    dp0, dp1 = int(c != '0'), int(c != '1')
                if stk[-1][2] == '&':
                    stk[-1] = [min(stk[-1][0], dp0),
                               min(stk[-1][1]+dp1, min(stk[-1][1], dp1)+1),
                               None]
                elif stk[-1][2] == '|':
                    stk[-1] = [min(stk[-1][0]+dp0, min(stk[-1][0], dp0)+1),
                               min(stk[-1][1], dp1),
                               None]
                else:  # operand
                    stk[-1] = [dp0, dp1, None]
            else:
                stk[-1][2] = c
        return max(stk[0][0], stk[0][1])

# 1900 Hard 1900 The Earliest and Latest Rounds Where Players Compete

In [ ]:
# Time:  O(n^2) states * O(n^2) per state = O(n^4)
# Space: O(n^2 + (n/2)^2 + (n/4)^2 + ... ) = O(n^2)

class Solution(object):
    def earliestAndLatest(self, n, firstPlayer, secondPlayer):
        """
        :type n: int
        :type firstPlayer: int
        :type secondPlayer: int
        :rtype: List[int]
        """
        def memoization(t, l, r, lookup):
            # t: total number of players,
            # l: number of players left to the nearest top2 player,
            # r: number of players right to the nearest top2 player
            if (t, l, r) not in lookup:
                if l == r:
                    return (1, 1)
                if l > r:  # make sure l <= r
                    l, r, = r, l
                result = [float("inf"), 0]
                for i in xrange(l+1):
                    l_win_cnt, l_lose_cnt, nt, pair_cnt = i+1, l-i, (t+1)//2, t//2
                    min_j = max(l_lose_cnt, r-(pair_cnt-l_lose_cnt))  # j >= l_lose_cnt and j >= r-(pair_cnt-l_lose_cnt)
                    max_j = min(r-l_win_cnt, (nt-l_win_cnt)-1)  # j <= r-l_win_cnt and j <= (nt-l_win_cnt)-1
                    for j in xrange(min_j, max_j+1):
                        tmp = memoization(nt, i, j, lookup)
                        result = min(result[0], tmp[0]+1), max(result[1], tmp[1]+1)
                lookup[t, l, r] = result
            return lookup[t, l, r]
        
        return memoization(n, firstPlayer-1, n-secondPlayer, {})

# 1912 Hard 1912 Design Movie Rental System

In [ ]:
# Time:  ctor:   O(nlogn)
#        search: O(logn)
#        rent:   O(logn)
#        drop:   O(logn)
#        report: O(logn)
# Space: O(n)

import collections
from sortedcontainers import SortedList


class MovieRentingSystem(object):

    def __init__(self, n, entries):
        """
        :type n: int
        :type entries: List[List[int]]
        """
        self.__movie_to_ordered_price_shop = collections.defaultdict(SortedList) 
        self.__shop_movie_to_price = {}
        self.__rented_ordered_price_shop_movie = SortedList()
        for s, m, p in entries:
            self.__movie_to_ordered_price_shop[m].add((p, s))
            self.__shop_movie_to_price[s, m] = p

    def search(self, movie):
        """
        :type movie: int
        :rtype: List[int]
        """
        return [s for _, s in self.__movie_to_ordered_price_shop[movie][:5]]

    def rent(self, shop, movie):
        """
        :type shop: int
        :type movie: int
        :rtype: None
        """
        price = self.__shop_movie_to_price[shop, movie]
        self.__movie_to_ordered_price_shop[movie].remove((price, shop))
        self.__rented_ordered_price_shop_movie.add((price, shop, movie))

    def drop(self, shop, movie):
        """
        :type shop: int
        :type movie: int
        :rtype: None
        """
        price = self.__shop_movie_to_price[shop, movie]
        self.__movie_to_ordered_price_shop[movie].add((price, shop))
        self.__rented_ordered_price_shop_movie.remove((price, shop, movie))

    def report(self):
        """
        :rtype: List[List[int]]
        """
        return [[s, m] for _, s, m in self.__rented_ordered_price_shop_movie[:5]]

# 1916 Hard 1916 Count Ways to Build Rooms in an Ant Colony

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def waysToBuildRooms(self, prevRoom):
        """
        :type prevRoom: List[int]
        :rtype: int
        """
        MOD = 10**9+7
        fact = [1, 1]
        inv = [0, 1]
        inv_fact = [1, 1]
        def nCr(n, k):
            while len(inv) <= n:  # lazy initialization
                fact.append(fact[-1]*len(inv) % MOD)
                inv.append(inv[MOD%len(inv)]*(MOD-MOD//len(inv)) % MOD)  # https://cp-algorithms.com/algebra/module-inverse.html
                inv_fact.append(inv_fact[-1]*inv[-1] % MOD)
            return (fact[n]*inv_fact[n-k] % MOD) * inv_fact[k] % MOD

        def dfs(adj, curr):
            total_ways, total_cnt = 1, 0
            for child in adj[curr]:
                ways, cnt = dfs(adj, child)
                total_cnt += cnt
                total_ways = (((total_ways*ways) % MOD)*nCr(total_cnt, cnt)) % MOD
            return total_ways, total_cnt+1

        adj = [[] for _ in xrange(len(prevRoom))]
        for i in xrange(1, len(prevRoom)):
            adj[prevRoom[i]].append(i)
        return dfs(adj, 0)[0]

# 1923 Hard 1923 Longest Common Subpath

In [ ]:
# Time:  O(m * nlogn)
# Space: O(n)

class Solution(object):
    def longestCommonSubpath(self, n, paths):
        """
        :type n: int
        :type paths: List[List[int]]
        :rtype: int
        """
        def RabinKarp(arr, x):  # double hashing
            hashes = tuple([reduce(lambda h,x: (h*p+x)%MOD, (arr[i] for i in xrange(x)), 0) for p in P])
            powers = [pow(p, x, MOD) for p in P]
            lookup = {hashes}
            for i in xrange(x, len(arr)):
                hashes = tuple([(hashes[j]*P[j] - arr[i-x]*powers[j] + arr[i])%MOD for j in xrange(len(P))])  # in smaller datasets, tuple from list is much faster than tuple from generator, see https://stackoverflow.com/questions/16940293/why-is-there-no-tuple-comprehension-in-python
                lookup.add(hashes)
            return lookup
        
        def check(paths, x):
            intersect = RabinKarp(paths[0], x)
            for i in xrange(1, len(paths)):
                intersect = set.intersection(intersect, RabinKarp(paths[i], x))
                if not intersect:
                    return False
            return True

        MOD, P = 10**9+7, (113, 109)  # MOD could be the min prime of 7-digit number (10**6+3), P could be (2, 3)
        left, right = 1, min(len(p) for p in paths)
        while left <= right:
            mid = left + (right-left)//2
            if not check(paths, mid):
                right = mid-1
            else:
                left = mid+1
        return right


# Time:  O(m * nlogn)
# Space: O(n)
class Solution2(object):
    def longestCommonSubpath(self, n, paths):
        """
        :type n: int
        :type paths: List[List[int]]
        :rtype: int
        """
        def RabinKarp(arr, x):
            h = reduce(lambda h,x: (h*P+x)%MOD, (arr[i] for i in xrange(x)), 0)
            power = pow(P, x, MOD)
            lookup = {h}
            for i in xrange(x, len(arr)):
                h = (h*P - arr[i-x]*power + arr[i])%MOD
                lookup.add(h)
            return lookup
        
        def check(paths, x):
            intersect = RabinKarp(paths[0], x)
            for i in xrange(1, len(paths)):
                intersect = set.intersection(intersect, RabinKarp(paths[i], x))
                if not intersect:
                    return False
            return True

        MOD, P = 10**11+19, max(x for p in paths for x in p)+1  # MOD is the min prime of 12-digit number
        left, right = 1, min(len(p) for p in paths)
        while left <= right:
            mid = left + (right-left)//2
            if not check(paths, mid):
                right = mid-1
            else:
                left = mid+1
        return right

# 1924 Hard 1924 Erect the Fence II

In [ ]:
# Time:  O(n) on average
# Space: O(n)

import random


# reference: https://en.wikipedia.org/wiki/Smallest-circle_problem
class Solution(object):
    def outerTrees(self, trees):
        """
        :type trees: List[List[int]]
        :rtype: List[float]
        """
        def dist(a, b):
            return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5

        def inside(c, p):
            return dist(c[0], p) < c[1]+EPS

        def circle_center(bx, by, cx, cy):
            B = bx*bx + by*by
            C = cx*cx + cy*cy
            D = bx*cy - by*cx
            return [float(cy*B - by*C)/(2*D),
                    float(bx*C - cx*B)/(2*D)]

        def circle_from_2_points(A, B):
            C = [(A[0]+B[0])/2.0, (A[1]+B[1])/2.0]
            return [C, dist(A, B)/2.0]

        def circle_from_3_points(A, B, C):
            I = circle_center(B[0]-A[0], B[1]-A[1],
                              C[0]-A[0], C[1]-A[1])
            I[0] += A[0]
            I[1] += A[1]
            return [I, dist(I, A)]

        def trivial(boundaries):  # circumscribed circle
            if not boundaries:
                return None
            if len(boundaries) == 1:
                return [boundaries[0], 0.0]
            if len(boundaries) == 2:
                return circle_from_2_points(boundaries[0], boundaries[1])
            return circle_from_3_points(boundaries[0], boundaries[1], boundaries[2])

        def Welzl(points, boundaries, curr):
            if curr == len(points) or len(boundaries) == 3:
                return trivial(boundaries)
            result = Welzl(points, boundaries, curr+1)
            if result is not None and inside(result, points[curr]):
                return result
            boundaries.append(points[curr])
            result = Welzl(points, boundaries, curr+1)
            boundaries.pop()
            return result

        EPS = 1e-5
        random.seed(0)
        random.shuffle(trees)
        result = Welzl(trees, [], 0)
        return result[0][0], result[0][1], result[1]

# 1928 Hard 1928 Minimum Cost to Reach Destination in Time

In [ ]:
# Time:  O((|E| + |V|) * log|V|) = O(|E| * log|V|),
#        if we can further to use Fibonacci heap, it would be O(|E| + |V| * log|V|)
# Space: O(|E| + |V|) = O(|E|)

import collections
import heapq


# Dijkstra's algorithm
class Solution(object):
    def minCost(self, maxTime, edges, passingFees):
        """
        :type maxTime: int
        :type edges: List[List[int]]
        :type passingFees: List[int]
        :rtype: int
        """        
        adj = [[] for i in xrange(len(passingFees))]
        for u, v, w in edges:
            adj[u].append((v, w))
            adj[v].append((u, w))
        best = collections.defaultdict(lambda:float("inf"))
        best[0] = 0
        min_heap = [(passingFees[0], 0, 0)]
        while min_heap:
            result, u, w = heapq.heappop(min_heap)
            if w > maxTime:  # state with best[u] < w can't be filtered, which may have less cost
                continue
            if u == len(passingFees)-1:
                return result
            for v, nw in adj[u]:
                if w+nw < best[v]:  # from less cost to more cost, only need to check state with less time
                    best[v] = w+nw
                    heapq.heappush(min_heap, (result+passingFees[v], v, w+nw))
        return -1

# 1931 Hard 1931 Painting a Grid With Three Different Colors

In [ ]:
# Time:  O(m * 2^m + 3^m + 2^(3 * m) * logn) = O(2^(3 * m) * logn)
# Space: O(2^(2 * m))

import collections
import itertools


# better complexity for small m, super large n
# matrix exponentiation solution
class Solution(object):
    def colorTheGrid(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        """
        MOD = 10**9+7
        def backtracking(mask1, mask2, basis, result):  # Time: O(2^m), Space: O(2^m)
            if not basis:
                result.append(mask2)
                return
            for i in xrange(3):
                if (mask1 == -1 or mask1//basis%3 != i) and (mask2 == -1 or mask2//(basis*3)%3 != i):
                    backtracking(mask1, mask2+i*basis if mask2 != -1 else i*basis, basis//3, result)

        def matrix_mult(A, B):
            ZB = zip(*B)
            return [[sum(a*b % MOD for a, b in itertools.izip(row, col)) % MOD for col in ZB] for row in A]
 
        def matrix_expo(A, K):
            result = [[int(i == j) for j in xrange(len(A))] for i in xrange(len(A))]
            while K:
                if K % 2:
                    result = matrix_mult(result, A)
                A = matrix_mult(A, A)
                K /= 2
            return result

        def normalize(basis, mask):
            norm = {}
            result = 0
            while basis:
                x = mask//basis%3
                if x not in norm:
                    norm[x] = len(norm)
                result += norm[x]*basis
                basis //= 3
            return result

        if m > n:
            m, n = n, m
        basis = 3**(m-1)
        masks = []
        backtracking(-1, -1, basis, masks)  # Time: O(2^m), Space: O(2^m)
        assert(len(masks) == 3 * 2**(m-1))
        lookup = {mask:normalize(basis, mask) for mask in masks}  # Time: O(m * 2^m)
        normalized_mask_cnt = collections.Counter(lookup[mask] for mask in masks)
        assert(len(normalized_mask_cnt) == 3*2**(m-1) // 3 // (2 if m >= 2 else 1))  # divided by 3 * 2 is since the first two colors are normalized to speed up performance
        adj = collections.defaultdict(list)
        for mask in normalized_mask_cnt.iterkeys():  # O(3^m) leaves which are all in depth m => Time: O(3^m), Space: O(3^m)
            backtracking(mask, -1, basis, adj[mask])
        normalized_adj = collections.defaultdict(lambda:collections.defaultdict(int))
        for mask1, masks2 in adj.iteritems():
            for mask2 in masks2:
                normalized_adj[mask1][lookup[mask2]] = (normalized_adj[mask1][lookup[mask2]]+1)%MOD
        # divided by 3 * 2 is since the first two colors in upper row are normalized to speed up performance,
        # since first two colors in lower row which has at most 3 choices could be also normalized, lower bound is upper bound divided by at most 3
        assert(2*3**m // 3 // 2 // 3 <= sum(len(v) for v in normalized_adj.itervalues()) <= 2*3**m // 3 // 2)
        return reduce(lambda x,y: (x+y)%MOD,
                      matrix_mult([normalized_mask_cnt.values()],
                                   matrix_expo([[normalized_adj[mask1][mask2]
                                                 for mask2 in normalized_mask_cnt.iterkeys()] 
                                                 for mask1 in normalized_mask_cnt.iterkeys()], n-1))[0],
                      0)  # Time: O((2^m)^3 * logn), Space: O((2^m)^2)


# Time:  O(n * 3^m)
# Space: O(3^m)
import collections


# better complexity for small m, large n
class Solution2(object):
    def colorTheGrid(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        """
        MOD = 10**9+7
        def find_masks(m, basis):  # Time: 3 + 3*2 + 3*2*2 + ... + 3*2^(m-1) = 3 * (2^m - 1) = O(2^m), Space: O(2^m)
            masks = [0]
            for c in xrange(m):
                new_masks = []
                for mask in masks:
                    choices = {0, 1, 2}
                    if c > 0:
                        choices.discard(mask//basis)  # get left grid
                    for x in choices:
                        new_masks.append((x*basis)+(mask//3))  # encoding mask
                masks = new_masks
            return masks

        def find_adj(m, basis, dp):
            # Time:  3*2^(m-1) * (1 + 2 + 2 * (3/2) + 2 * (3/2)^2 + ... + 2 * (3/2)^(m-2)) =
            #        3*2^(m-1) * (1+2*((3/2)^(m-1)-1)/((3/2)-1)) =
            #        3*2^(m-1) * (1+4*((3/2)^(m-1)-1)) =
            #        3*2^(m-1) * (4*(3/2)^(m-1)-3) =
            #        4*3^m-9*2^(m-1) =
            #        O(3^m),
            # Space: O(3^m)
            adj = collections.defaultdict(list)
            for mask in dp.iterkeys():  # O(2^m)
                adj[mask].append(mask)
            for c in xrange(m):
                assert(sum(len(v) for v in adj.itervalues()) == (3**c * 2**(m-(c-1)) if c >= 1 else 3 * 2**(m-1)) // 3 // (2 if m >= 2 else 1))  # divided by 3 * 2 is since the first two colors are normalized to speed up performance
                new_adj = collections.defaultdict(list)
                for mask1, mask2s in adj.iteritems():
                    for mask in mask2s:
                        choices = {0, 1, 2}
                        choices.discard(mask%3)  # get up grid
                        if c > 0:
                            choices.discard(mask//basis)  # get left grid
                        for x in choices:
                            new_adj[mask1].append((x*basis)+(mask//3))  # encoding mask
                adj = new_adj
            assert(sum(3**c * 2**(m-(c-1)) if c >= 1 else 3 * 2**(m-1) for c in xrange(m)) == 4*3**m-9*2**(m-1))
            return adj
 
        def normalize(basis, mask):
            norm = {}
            result = 0
            while basis:
                x = mask//basis%3
                if x not in norm:
                    norm[x] = len(norm)
                result += norm[x]*basis
                basis //= 3
            return result

        if m > n:
            m, n = n, m
        basis = 3**(m-1)
        masks = find_masks(m, basis)  # alternative of backtracking, Time: O(2^m), Space: O(2^m)
        assert(len(masks) == 3 * 2**(m-1))
        lookup = {mask:normalize(basis, mask) for mask in masks}  # Time: O(m * 2^m)
        dp = collections.Counter(lookup[mask] for mask in masks)  # normalize colors to speed up performance
        adj = find_adj(m, basis, dp)  # alternative of backtracking, Time: O(3^m), Space: O(3^m)
        # proof:
        #   'o' uses the same color with its bottom-left one, 
        #   'x' uses the remaining color different from its left one and bottom-left one,
        #   k is the cnt of 'o', 
        #     [3, 1(o), 1(x), 1(o), ..., 1(o), 1(x)] => nCr(m-1, k) * 3 * 2 * 2^k for k in xrange(m) = 3 * 2 * (2+1)^(m-1) = 2*3^m combinations
        #     [2,    2,    1,    2, ...,  2,      1]
        # another proof:
        #   given previous pair of colors, each pair of '?' has 3 choices of colors
        #     [3, ?, ?, ..., ?] => 3 * 2 * 3^(m-1) = 2*3^m combinations
        #         |  |       |
        #         3  3       3
        #         |  |       |
        #     [2, ?, ?, ..., ?]
        normalized_adj = collections.defaultdict(lambda:collections.defaultdict(int))
        for mask1, mask2s in adj.iteritems():
            for mask2 in mask2s:
                normalized_adj[lookup[mask1]][lookup[mask2]] = (normalized_adj[lookup[mask1]][lookup[mask2]]+1)%MOD
        # divided by 3 * 2 is since the first two colors in upper row are normalized to speed up performance,
        # since first two colors in lower row which has at most 3 choices could be also normalized, lower bound is upper bound divided by at most 3
        assert(2*3**m // 3 // 2 // 3 <= sum(len(v) for v in normalized_adj.itervalues()) <= 2*3**m // 3 // 2)
        for _ in xrange(n-1):  # Time: O(n * 3^m), Space: O(2^m)
            assert(len(dp) == 3*2**(m-1) // 3 // (2 if m >= 2 else 1))  # divided by 3 * 2 is since the first two colors are normalized to speed up performance
            new_dp = collections.Counter()
            for mask, v in dp.iteritems():
                for new_mask, cnt in normalized_adj[mask].iteritems():
                    new_dp[lookup[new_mask]] = (new_dp[lookup[new_mask]] + v*cnt) % MOD
            dp = new_dp
        return reduce(lambda x,y: (x+y)%MOD, dp.itervalues(), 0)  # Time: O(2^m)


# Time:  (m * n grids) * (O(3*3*2^(m-2)) possible states per grid) = O(n * m * 2^m)
# Space: O(3*3*2^(m-2)) = O(2^m)
import collections


# better complexity for large m, large n
class Solution3(object):
    def colorTheGrid(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        """
        MOD = 10**9+7
        def normalize(basis, mask, lookup):  # compute and cache, at most O(3*2^(m-3)) time and space
            if mask not in lookup[basis]:
                norm = {}
                result, b = 0, basis
                while b:
                    x = mask//b%3
                    if x not in norm:
                        norm[x] = len(norm)
                    result += norm[x]*b
                    b //= 3
                lookup[basis][mask] = result
            return lookup[basis][mask]

        if m > n:
            m, n = n, m
        basis = b = 3**(m-1)
        lookup = collections.defaultdict(dict)
        dp = collections.Counter({0: 1})
        for idx in xrange(m*n):
            r, c = divmod(idx, m)
            # sliding window with size m doesn't cross rows:
            #   [3, 2, ..., 2] => 3*2^(m-1) combinations
            assert(r != 0 or c != 0 or len(dp) == 1)
            assert(r != 0 or c == 0 or len(dp) == 3*2**(c-1) // 3 // (2 if c >= 2 else 1))  # divided by 3 * 2 is since the first two colors are normalized to speed up performance
            assert(r == 0 or c != 0 or len(dp) == 3*2**(m-1) // 3 // (2 if m >= 2 else 1))  # divided by 3 * 2 is since the first two colors are normalized to speed up performance
            # sliding window with size m crosses rows:
            #   [*, ..., *, *, 3, 2, ..., 2] => 3*3 * 2^(m-2) combinations
            #   [2, ..., 2, 3, *, *, ..., *]
            assert(r == 0 or c == 0 or len(dp) == (1 if m == 1 else 2 if m == 2 else 3*3 * 2**(m-2) // 3 // 2))  # divided by 3 * 2 for m >= 3 is since the first two colors of window are normalized to speed up performance
            new_dp = collections.Counter()
            for mask, v in dp.iteritems():
                choices = {0, 1, 2}
                if r > 0:
                    choices.discard(mask%3)  # get up grid
                if c > 0:
                    choices.discard(mask//basis)  # get left grid
                for x in choices:
                    new_mask = normalize(basis//b, ((x*basis)+(mask//3))//b, lookup)*b  # encoding mask
                    new_dp[new_mask] = (new_dp[new_mask]+v)%MOD
            if b > 1:
                b //= 3
            dp = new_dp
        return reduce(lambda x,y: (x+y)%MOD, dp.itervalues(), 0)  # Time: O(2^m)

# 1932 Hard 1932 Merge BSTs to Create Single BST

In [ ]:
# Time:  O(n)
# Space: O(n)

# Definition for a binary tree node.
class TreeNode(object):
    def __init__(self, val=0, left=None, right=None):
        pass


class Solution(object):
    def canMerge(self, trees):
        """
        :type trees: List[TreeNode]
        :rtype: TreeNode
        """
        def find_leaves_and_roots(trees, leaf_vals_set, val_to_root):
            for root in trees:
                val_to_root[root.val] = root
                q = [root]
                while q:
                    new_q = []
                    for node in q:
                        if node.left is None and node.right is None:
                            if node is not root:
                                leaf_vals_set.add(node.val)
                            continue
                        if node.left:
                            new_q.append(node.left)
                        if node.right:
                            new_q.append(node.right)
                    q = new_q

        def find_root(trees, left_vals_set, val_to_root):
            root = None
            for node in trees:
                if node.val in leaf_vals_set:
                    continue
                if root:  # multiple roots
                    return None
                root = node
            return root

        def merge_bsts(root, left_vals_set, val_to_root):
            if not root:
                return None
            del val_to_root[root.val]
            q = [(root, float("-inf"), float("inf"))]
            while q:
                new_q = []
                for node, left, right in q:
                    if not (left < node.val < right):
                        return None
                    if node.left:
                        if node.left.val in leaf_vals_set and node.left.val in val_to_root:
                            node.left = val_to_root[node.left.val]
                            del val_to_root[node.left.val]
                        new_q.append((node.left, left, node.val))
                    if node.right:
                        if node.right.val in leaf_vals_set and node.right.val in val_to_root:
                            node.right = val_to_root[node.right.val]
                            del val_to_root[node.right.val]
                        new_q.append((node.right, node.val, right))
                q = new_q
            return root if not val_to_root else None

        leaf_vals_set, val_to_root = set(), {}
        find_leaves_and_roots(trees, leaf_vals_set, val_to_root)    
        root = find_root(trees, leaf_vals_set, val_to_root)
        return merge_bsts(root, leaf_vals_set, val_to_root)

# 1938 Hard 1938 Maximum Genetic Difference Query

In [ ]:
# Time:  O(nlogk + mlogk), k is max(max(vals), n-1)
# Space: O(n + logk)

import collections


class Trie(object):
    def __init__(self, bit_count):
        self.__root = {}
        self.__bit_count = bit_count
        
    def insert(self, num, v):
        node = self.__root
        for i in reversed(xrange(self.__bit_count)):
            curr = (num>>i) & 1
            new_node = node.setdefault(curr, collections.defaultdict(int))
            new_node["_cnt"] += v
            if not new_node["_cnt"]:
                del node[curr]
                break
            node = new_node
                
    def query(self, num):
        node, result = self.__root, 0
        for i in reversed(xrange(self.__bit_count)):
            curr = (num>>i) & 1
            if 1^curr in node:
                node = node[1^curr]
                result |= 1<<i
            else:
                node = node[curr]
        return result


class Solution(object):
    def maxGeneticDifference(self, parents, queries):
        """
        :type parents: List[int]
        :type queries: List[List[int]]
        :rtype: List[int]
        """
        def iter_dfs(adj, qs, trie, result):
            stk = [(1, adj[-1][0])]
            while stk:
                step, node = stk.pop()
                if step == 1:
                    trie.insert(node, 1)
                    for i, val in qs[node]:
                        result[i] = trie.query(val)
                    stk.append((2, node))
                    for child in reversed(adj[node]):
                        stk.append((1, child))
                elif step == 2:
                    trie.insert(node, -1)
    
        adj = collections.defaultdict(list)
        for node, parent in enumerate(parents):
            adj[parent].append(node)
        qs = collections.defaultdict(list)
        max_val = len(parents)-1
        for i, (node, val) in enumerate(queries):
            qs[node].append((i, val))
            max_val = max(max_val, val)
        result = [0]*len(queries)
        iter_dfs(adj, qs, Trie(max_val.bit_length()), result)
        return result


# Time:  O(nlogk + mlogk), k is max(max(vals), n-1)
# Space: O(n + logk)
import collections


class Trie(object):
    def __init__(self, bit_count):
        self.__root = {}
        self.__bit_count = bit_count
        
    def insert(self, num, v):
        node = self.__root
        for i in reversed(xrange(self.__bit_count)):
            curr = (num>>i) & 1
            new_node = node.setdefault(curr, collections.defaultdict(int))
            new_node["_cnt"] += v
            if not new_node["_cnt"]:
                del node[curr]
                break
            node = new_node
                
    def query(self, num):
        node, result = self.__root, 0
        for i in reversed(xrange(self.__bit_count)):
            curr = (num>>i) & 1
            if 1^curr in node:
                node = node[1^curr]
                result |= 1<<i
            else:
                node = node[curr]
        return result


class Solution2(object):
    def maxGeneticDifference(self, parents, queries):
        """
        :type parents: List[int]
        :type queries: List[List[int]]
        :rtype: List[int]
        """
        def dfs(adj, qs, node, trie, result):
            trie.insert(node, 1)
            for i, val in qs[node]:
                result[i] = trie.query(val)
            for child in adj[node]:
                dfs(adj, qs, child, trie, result)
            trie.insert(node, -1)

        adj = collections.defaultdict(list)
        for node, parent in enumerate(parents):
            adj[parent].append(node)
        qs = collections.defaultdict(list)
        max_val = len(parents)-1
        for i, (node, val) in enumerate(queries):
            qs[node].append((i, val))
            max_val = max(max_val, val)
        result = [0]*len(queries)
        dfs(adj, qs, adj[-1][0], Trie(max_val.bit_length()), result)
        return result

# 1944 Hard 1944 Number of Visible People in a Queue

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def canSeePersonsCount(self, heights):
        """
        :type heights: List[int]
        :rtype: List[int]
        """
        result = [0]*len(heights)
        stk = []
        for i, h in enumerate(heights):
            while stk and heights[stk[-1]] < h:
                result[stk.pop()] += 1
            if stk:
                result[stk[-1]] += 1
            if stk and heights[stk[-1]] == h:
                stk.pop()
            stk.append(i)
        return result


# Time:  O(n)
# Space: O(n)
class Solution2(object):
    def canSeePersonsCount(self, heights):
        """
        :type heights: List[int]
        :rtype: List[int]
        """
        result = [0]*len(heights)
        stk = []
        for i in reversed(xrange(len(heights))):
            cnt = 0
            while stk and heights[stk[-1]] < heights[i]:
                stk.pop()
                cnt += 1
            result[i] = cnt+1 if stk else cnt
            if stk and heights[stk[-1]] == heights[i]:
                stk.pop()
            stk.append(i)
        return result

# 1948 Hard 1948 Delete Duplicate Folders in System

In [ ]:
class Node(object):
    def __init__(self, val):
        self.val = val
        self.key = None
        self.children = {}

class Solution(object):
    def deleteDuplicateFolder(self, paths):
        def setKey(node):
            node.key = ''
            for c in sorted(node.children.keys()): #need to be sorted. so when child structs are the same, we won't generate different key from different iteration order.
                setKey(node.children[c])
                node.key += node.children[c].val + '|' + node.children[c].key + '|' #generate a key for each node. only considering its children structure. (see the "identical" definition, it does not consider the val of the node itself.)
                
            keyCount[node.key] += 1
        
        def addPath(node, path):
            if node.children and keyCount[node.key]>1: return #leaf node does not apply to this rule
            ans.append(path+[node.val])
            for c in node.children:
                addPath(node.children[c], path+[node.val])
            
            
        ans = []
        root = Node('/')
        keyCount = collections.Counter()
        
        #build the tree
        for path in paths:
            node = root
            for c in path:
                if c not in node.children: node.children[c] = Node(c)
                node = node.children[c]
        
        #set all nodes key recursively
        setKey(root)

        #build ans
        for c in root.children:
            addPath(root.children[c], [])
        
        return ans

# 1955 Hard 1955 Count Number of Special Subsequences

In [ ]:
# Time:  O(n)
# Space: O(1)

class Solution(object):
    def countSpecialSubsequences(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        MOD = 10**9+7
        dp = [0]*3
        for x in nums:
            dp[x] = ((dp[x]+dp[x])%MOD+(dp[x-1] if x-1 >= 0 else 1))%MOD
        return dp[-1]

# 1956 Hard 1956 Minimum Time For K Virus Variants to Spread

In [ ]:
# Time:  O(nlogn * logr), r is the sum of range x size and range y size
# Space: O(n)

# Range Maximum Query
class SegmentTree(object):  # 0-based index
    def __init__(self, N,
                 build_fn=lambda x, y: [y]*(2*x),
                 query_fn=lambda x, y: y if x is None else max(x, y),
                 update_fn=lambda x, y: y if x is None else x+y,
                 default_val=0):
        self.N = N
        self.H = (N-1).bit_length()
        self.query_fn = query_fn
        self.update_fn = update_fn
        self.default_val = default_val
        self.tree = build_fn(N, default_val)
        self.lazy = [None]*N

    def __apply(self, x, val):
        self.tree[x] = self.update_fn(self.tree[x], val)
        if x < self.N:
            self.lazy[x] = self.update_fn(self.lazy[x], val)

    def update(self, L, R, h):  # Time: O(logN), Space: O(N)
        def pull(x):
            while x > 1:
                x //= 2
                self.tree[x] = self.query_fn(self.tree[x*2], self.tree[x*2+1])
                if self.lazy[x] is not None:
                    self.tree[x] = self.update_fn(self.tree[x], self.lazy[x])

        L += self.N
        R += self.N
        L0, R0 = L, R
        while L <= R:
            if L & 1:  # is right child
                self.__apply(L, h)
                L += 1
            if R & 1 == 0:  # is left child
                self.__apply(R, h)
                R -= 1
            L //= 2
            R //= 2
        pull(L0)
        pull(R0)

    def query(self, L, R):  # Time: O(logN), Space: O(N)
        def push(x):
            n = 2**self.H
            while n != 1:
                y = x // n
                if self.lazy[y] is not None:
                    self.__apply(y*2, self.lazy[y])
                    self.__apply(y*2 + 1, self.lazy[y])
                    self.lazy[y] = None
                n //= 2

        result = None
        if L > R:
            return result

        L += self.N
        R += self.N
        push(L)
        push(R)
        while L <= R:
            if L & 1:  # is right child
                result = self.query_fn(result, self.tree[L])
                L += 1
            if R & 1 == 0:  # is left child
                result = self.query_fn(result, self.tree[R])
                R -= 1
            L //= 2
            R //= 2
        return result
    
    def __str__(self):
        showList = []
        for i in xrange(self.N):
            showList.append(self.query(i, i))
        return ",".join(map(str, showList))


# competitive programming solution
class Solution(object):
    def minDayskVariants(self, points, k):
        """
        :type points: List[List[int]]
        :type k: int
        :rtype: int
        """
        def add_rec(rec, intervals):
            x0, y0, x1, y1 = rec
            # add [y0, y1] by 1 in [x0, x1+1)
            intervals.append([[x0,   +1], [y0, y1]])
            intervals.append([[x1+1, -1], [y0, y1]])

        def check(points, k, l):  # Time: O(nlogn), Space: O(n)
            intervals = []
            y_set = set()
            for x, y in points:
                add_rec([x-l, y-l, x+l, y+l], intervals)
                y_set.add(y-l)
                y_set.add(y+l)
            intervals.sort()
            y_to_idx = {y:i for i, y in enumerate(sorted(y_set))}  # coordinate compression
            st = SegmentTree(len(y_to_idx))
            for [_, v], [y0, y1] in intervals:  # line sweep
                st.update(y_to_idx[y0], y_to_idx[y1], v)
                if st.query(0, len(y_to_idx)-1) >= k:
                    return True
            return False
                
        points = [[x+y, x-y] for x, y in points]  # rotate
        min_x = min(points)[0]
        max_x = max(points)[0]
        min_y = min(points, key=lambda x: x[1])[1]
        max_y = max(points, key=lambda x: x[1])[1]
        left, right = 0, ((max_x-min_x)+(max_y-min_y)+1)//2
        while left <= right:
            mid = left + (right-left)//2
            if check(points, k, mid):
                right = mid-1
            else:
                left = mid+1
        return left


# Time:  O(n^2 * logr), r is the sum of range x size and range y size
# Space: O(n)
import collections


# interview solution
class Solution2(object):
    def minDayskVariants(self, points, k):
        """
        :type points: List[List[int]]
        :type k: int
        :rtype: int
        """
        def add_rec(rec, intervals):
            x0, y0, x1, y1 = rec
            # add [y0, y1+1) by 1 in [x0, x1+1)
            intervals[x0][y0] += 1
            intervals[x0][y1+1] -= 1
            intervals[x1+1][y0] -= 1
            intervals[x1+1][y1+1] += 1

        def check(points, k, l):  # Time: O(n^2), Space: O(n)
            intervals = collections.defaultdict(lambda:collections.defaultdict(int))
            y_set = set()
            for x, y in points:
                add_rec([x-l, y-l, x+l, y+l], intervals)
                y_set.add(y-l)
                y_set.add(y+l+1)
            sorted_y = sorted(y_set)
            sorted_x = sorted(intervals.iterkeys())
            count = collections.Counter()
            for x in sorted_x:  # line sweep
                for y, c in intervals[x].iteritems():
                    count[y] += c
                cnt = 0
                for y in sorted_y:
                    cnt += count[y]
                    if cnt >= k:
                        return True
            return False
                
        points = [[x+y, x-y] for x, y in points]  # rotate
        min_x = min(points)[0]
        max_x = max(points)[0]
        min_y = min(points, key=lambda x: x[1])[1]
        max_y = max(points, key=lambda x: x[1])[1]
        left, right = 0, ((max_x-min_x)+(max_y-min_y)+1)//2
        while left <= right:
            mid = left + (right-left)//2
            if check(points, k, mid):
                right = mid-1
            else:
                left = mid+1
        return left

# 1960 Hard 1960 Maximum Product of the Length of Two Palindromic Substrings

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


class Solution(object):
    def maxProduct(self, s):
        """
        :type s: str
        :rtype: int
        """
        def manacher(s):
            s = '^#' + '#'.join(s) + '#$'
            P = [0]*len(s)
            C, R = 0, 0
            for i in xrange(1, len(s)-1):
                i_mirror = 2*C-i
                if R > i:
                    P[i] = min(R-i, P[i_mirror])
                while s[i+1+P[i]] == s[i-1-P[i]]:
                    P[i] += 1
                if i+P[i] > R:
                    C, R = i, i+P[i]
            return P

        P = manacher(s)
        q = collections.deque()
        left = [0]
        for i in xrange(len(s)):
            while q and q[0][1] < i:
                q.popleft()
            left.append(max(left[-1], 1+2*(i-q[0][0]) if q else 1))
            q.append((i, i+P[2*i+2]//2))
        q = collections.deque()
        result = right = 0
        for i in reversed(xrange(len(s))):
            while q and q[0][1] > i:
                q.popleft()
            right = max(right, 1+2*(q[0][0]-i) if q else 1)
            q.append((i, i-P[2*i+2]//2))
            result = max(result, left[i]*right)
        return result


# Time:  O(n)
# Space: O(n)
class Solution2(object):
    def maxProduct(self, s):
        """
        :type s: str
        :rtype: int
        """
        def manacher(s):
            s = '^#' + '#'.join(s) + '#$'
            P = [0]*len(s)
            C, R = 0, 0
            for i in xrange(1, len(s)-1):
                i_mirror = 2*C-i
                if R > i:
                    P[i] = min(R-i, P[i_mirror])
                while s[i+1+P[i]] == s[i-1-P[i]]:
                    P[i] += 1
                if i+P[i] > R:
                    C, R = i, i+P[i]
            return P

        import operator
        def accumulate(iterable, func=operator.add, initial=None):
            it = iter(iterable)
            total = initial
            if initial is None:
                try:
                    total = next(it)
                except StopIteration:
                    return
            yield total
            for element in it:
                total = func(total, element)
                yield total

        def fin_max_len(s):
            P = manacher(s)
            intervals = [[(i-2)//2-P[i]//2, (i-2)//2+P[i]//2] for i in xrange(2,len(P)-2, 2)]
            dp = [0]*len(s)
            for l, r in reversed(intervals): 
                dp[r] = r-l+1
            for i in reversed(xrange(len(s)-1)):
                dp[i] = max(dp[i], dp[i+1]-2)
            return list(accumulate(dp, max, 0))
        
        l1, l2 = fin_max_len(s), fin_max_len(s[::-1])[::-1]
        return max(x*y for x, y in itertools.izip(l1, l2))

# 1964 Hard 1964 Find the Longest Valid Obstacle Course at Each Position

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import bisect


# binary search solution
class Solution(object):
    def longestObstacleCourseAtEachPosition(self, obstacles):
        """
        :type obstacles: List[int]
        :rtype: List[int]
        """
        result, stk = [], []
        for x in obstacles:
            i = bisect.bisect_right(stk, x)
            result.append(i+1)
            if i == len(stk):
                stk.append(0)
            stk[i] = x
        return result
    

# Range Maximum Query
class SegmentTree(object):  # 0-based index
    def __init__(self, N,
                 build_fn=lambda x, y: [y]*(2*x),
                 query_fn=lambda x, y: y if x is None else max(x, y),  # (lambda x, y: y if x is None else min(x, y))
                 update_fn=lambda x, y: y,
                 default_val=0):
        self.N = N
        self.H = (N-1).bit_length()
        self.query_fn = query_fn
        self.update_fn = update_fn
        self.default_val = default_val
        self.tree = build_fn(N, default_val)
        self.lazy = [None]*N

    def __apply(self, x, val):
        self.tree[x] = self.update_fn(self.tree[x], val)
        if x < self.N:
            self.lazy[x] = self.update_fn(self.lazy[x], val)

    def update(self, L, R, h):  # Time: O(logN), Space: O(N)
        def pull(x):
            while x > 1:
                x //= 2
                self.tree[x] = self.query_fn(self.tree[x*2], self.tree[x*2+1])
                if self.lazy[x] is not None:
                    self.tree[x] = self.update_fn(self.tree[x], self.lazy[x])

        L += self.N
        R += self.N
        L0, R0 = L, R
        while L <= R:
            if L & 1:  # is right child
                self.__apply(L, h) 
                L += 1
            if R & 1 == 0:  # is left child
                self.__apply(R, h)
                R -= 1
            L //= 2
            R //= 2
        pull(L0)
        pull(R0)

    def query(self, L, R):  # Time: O(logN), Space: O(N)
        def push(x):
            n = 2**self.H
            while n != 1:
                y = x // n
                if self.lazy[y] is not None:
                    self.__apply(y*2, self.lazy[y])
                    self.__apply(y*2 + 1, self.lazy[y])
                    self.lazy[y] = None
                n //= 2

        result = None
        if L > R:
            return result

        L += self.N
        R += self.N
        push(L)
        push(R)
        while L <= R:
            if L & 1:  # is right child
                result = self.query_fn(result, self.tree[L])
                L += 1
            if R & 1 == 0:  # is left child
                result = self.query_fn(result, self.tree[R])
                R -= 1
            L //= 2
            R //= 2
        return result
    
    def __str__(self):
        showList = []
        for i in xrange(self.N):
            showList.append(self.query(i, i))
        return ",".join(map(str, showList))


# Time:  O(nlogn)
# Space: O(n)
# segment tree solution
class Solution2_TLE(object):
    def longestObstacleCourseAtEachPosition(self, obstacles):
        """
        :type obstacles: List[int]
        :rtype: List[int]
        """
        sorted_obstacles = sorted(set(obstacles))
        lookup = {x:i for i, x in enumerate(sorted_obstacles)}
        segment_tree = SegmentTree(len(lookup))
        result = []
        for x in obstacles:
            cnt = segment_tree.query(0, lookup[x])+1
            result.append(cnt)
            segment_tree.update(lookup[x], lookup[x], cnt)
        return result

# 1970 Hard 1970 Last Day Where You Can Still Cross

In [ ]:
# Time:  O(m * n + c *  Î±(c)) = O(m * n)
# Space: O(m * n)

class UnionFind(object):  # Time: O(n * Î±(n)), Space: O(n)
    def __init__(self, n):
        self.set = range(n)
        self.rank = [0]*n

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y):
        x_root, y_root = map(self.find_set, (x, y))
        if x_root == y_root:
            return False
        if self.rank[x_root] < self.rank[y_root]:  # union by rank
            self.set[x_root] = y_root
        elif self.rank[x_root] > self.rank[y_root]:
            self.set[y_root] = x_root
        else:
            self.set[y_root] = x_root
            self.rank[x_root] += 1
        return True


class Solution(object):
    def latestDayToCross(self, row, col, cells):
        """
        :type row: int
        :type col: int
        :type cells: List[List[int]]
        :rtype: int
        """
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        def index(n, i, j):
            return i*n+j

        start, end = row*col, row*col+1
        uf = UnionFind(row*col+2)
        lookup = [[False]*col for _ in xrange(row)]
        for i in reversed(xrange(len(cells))):
            r, c = cells[i]
            r, c = r-1, c-1
            for dr, dc in directions:
                nr, nc = r+dr, c+dc
                if not (0 <= nr < row and 0 <= nc < col and lookup[nr][nc]):
                    continue
                uf.union_set(index(col, r, c), index(col, nr, nc))
            if r == 0:
                uf.union_set(start, index(col, r, c))
            if r == row-1:
                uf.union_set(end, index(col, r, c))
            if uf.find_set(start) == uf.find_set(end):
                return i
            lookup[r][c] = True
        return -1

# 1977 Hard 1977 Number of Ways to Separate Numbers

In [ ]:
# Time:  O(n^2)
# Space: O(n^2)

class Solution(object):
    def numberOfCombinations(self, num):
        """
        :type num: str
        :rtype: int
        """
        MOD = 10**9+7
        def find_longest_common_prefix(num):
            lcp = [[0]*(len(num)+1) for _ in xrange(len(num)+1)]  # lcp[i][j]: longest length of the common prefix which starts at num[i], num[j]
            for i in reversed(xrange(len(lcp)-1)):
                for j in reversed(xrange(len(lcp[0])-1)):
                    if num[i] == num[j]:
                        lcp[i][j] = lcp[i+1][j+1]+1
            return lcp

        def is_less_or_equal_to_with_same_length(num, lcp, i, j, l):
            return lcp[i][j] >= l or num[i+lcp[i][j]] < num[j+lcp[i][j]]

        lcp = find_longest_common_prefix(num)
        dp = [[0]*len(num) for _ in xrange(len(num))]  # dp[i][l]: the count of numbers ending at num[i], where the length of the last number is l+1
        dp[0][0] = int(num[0] != '0')
        for i in xrange(1, len(num)):
            dp[i][i] = dp[i-1][i-1]
            if num[i] == '0':
                continue
            accu = 0
            for l in xrange(len(num)-i+1):
                ni = i+l-1
                dp[ni][l-1] = accu  # accumulated count where the length of the second to last number ending at num[i-1] is shorter than the length of the last number ending at num[i+l-1]
                if i-l < 0:
                    continue
                if num[i-l] != '0' and is_less_or_equal_to_with_same_length(num, lcp, i-l, i, l):
                    dp[ni][l-1] = (dp[ni][l-1] + dp[i-1][l-1]) % MOD
                accu = (accu + dp[i-1][l-1]) % MOD
        return reduce(lambda total, x: (total+x)%MOD, dp[-1], 0)

# 1982 Hard 1982 Find Array Given Subset Sums

In [ ]:
# Time:  O(n * 2^n), len(sums) = 2^n
# Space: O(1)

# [proof]
# - let d = sorted_sums[0]-sorted_sums[1] and d != -d (d = 0 is trival), where one of +d/-d is the smallest positive or largest negative number of the original solution of [S1, ..., S(2^n)]
# - given Sp-d = 0 for some p in [1, 2^n] and Sq-(-d) = 0 for some q in [1, 2^n]
#   assume d is a number of the original solution of [S1, ..., S(2^n)] (the proof where -d is a number of the original solution is vice versa)
#   let Sq = x1+...+xi where 1 <= i <= n-1
#   let [d]+[x1, ..., xi]+[x(i+1), ..., x(n-1)] be the original solution
#   => new_sums([S1, ..., S(2^n)], d)
#      = subset_sums([x1, ..., xi]+[x(i+1), ..., x(n-1)])
#   if we choose -d as a number of a solution of [S1, ..., S(2^n)]
#   => new_sums([S1, ..., S(2^n)], -d)
#      = new_sums([S1, ..., S(2^n)], -(x1+...+xi))
#      = subset_sums([(-x1), ..., (-xi)]+[x(i+1), ..., x(n-1)])
#      => [-d]+[(-x1), ..., (-xi)]+[x(i+1), ..., x(n-1)] is also a solution
#
# [conclusion]
# - if new_sums with +d/-d (including d = 0) both contain zero, we can choose either one
# - if only one of new_sums with +d/-d contains zero, we can only choose the one with zero since subset_sums must contain zero

# optimized from solution4 (not using dict), runtime: 1040 ms
class Solution(object):
    def recoverArray(self, n, sums):
        """
        :type n: int
        :type sums: List[int]
        :rtype: List[int]
        """
        sums.sort()  # Time: O(2^n * log(2^n)) = O(n * 2^n)
        shift, l = 0, len(sums)
        result = []
        for _ in xrange(n):  # log(2^n) times, each time costs O(2^(n-len(result))), Total Time: O(2^n)
            new_shift = sums[0]-sums[1]
            assert(new_shift <= 0)
            has_zero, j, k = False, 0, 0
            for i in xrange(l):
                if k < j and sums[k] == sums[i]:  # skip shifted one
                    k += 1
                else:
                    if shift == sums[i]-new_shift:
                        has_zero = True
                    sums[j] = sums[i]-new_shift
                    j += 1
            if has_zero:  # contain 0, choose this side
                result.append(new_shift)
            else:  # contain no 0, choose another side and shift 0 offset
                result.append(-new_shift)
                shift -= new_shift
            l //= 2
        return result


# Time:  O(2^n + n * r), len(sums) = 2^n
#                      , r = max(sums)-min(sums)
# Space: O(2^n + r)
import collections


# optimized from solution4 (not using dict), runtime: 968 ms
class Solution2(object):
    def recoverArray(self, n, sums):
        """
        :type n: int
        :type sums: List[int]
        :rtype: List[int]
        """
        min_sum, max_sum = min(sums), max(sums)
        dp = [0]*(max_sum-min_sum+1)
        for x in sums:
            dp[x-min_sum] += 1
        sorted_sums = [x for x in xrange(min_sum, max_sum+1) if dp[x-min_sum]]  # Time: O(r)
        shift = 0
        result = []
        for _ in xrange(n):  # log(2^n) times, each time costs O(2^(n-len(result)))+O(r), Total Time: O(2^n + n * r)
            new_dp = [0]*(max_sum-min_sum+1)
            new_sorted_sums = []
            new_shift = sorted_sums[0]-sorted_sums[1] if dp[sorted_sums[0]-min_sum] == 1 else 0
            assert(new_shift <= 0)
            for x in sorted_sums:
                if not dp[x-min_sum]:
                    continue
                dp[(x-new_shift)-min_sum] -= dp[x-min_sum] if new_shift else dp[x-min_sum]//2
                new_dp[(x-new_shift)-min_sum] = dp[x-min_sum]
                new_sorted_sums.append(x-new_shift)
            dp = new_dp
            sorted_sums = new_sorted_sums
            if dp[shift-min_sum]:  # contain 0, choose this side
                result.append(new_shift)
            else:  # contain no 0, choose another side and shift 0 offset
                result.append(-new_shift)
                shift -= new_shift
        return result


# Time:  O(n * 2^n), len(sums) = 2^n
# Space: O(2^n)
import collections
import operator


# optimized from solution4, runtime: 1044 ms
class Solution3(object):
    def recoverArray(self, n, sums):
        """
        :type n: int
        :type sums: List[int]
        :rtype: List[int]
        """
        dp = {k: v for k, v in collections.Counter(sums).iteritems()}
        total = reduce(operator.ior, dp.itervalues(), 0)
        basis = total&-total  # find rightmost bit 1
        if basis > 1:
            for k in dp.iterkeys():
                dp[k] //= basis
        sorted_sums = sorted(dp.iterkeys())  # Time: O(2^n * log(2^n)) = O(n * 2^n)
        shift = 0
        result = [0]*(basis.bit_length()-1)
        for _ in xrange(n-len(result)):  # log(2^n) times, each time costs O(2^(n-len(result))), Total Time: O(2^n)
            new_dp = {}
            new_sorted_sums = []
            new_shift = sorted_sums[0]-sorted_sums[1]
            assert(new_shift < 0)
            for x in sorted_sums:
                if not dp[x]:
                    continue
                dp[x-new_shift] -= dp[x]
                new_dp[x-new_shift] = dp[x]
                new_sorted_sums.append(x-new_shift)
            dp = new_dp
            sorted_sums = new_sorted_sums
            if shift in dp:  # contain 0, choose this side
                result.append(new_shift)
            else:  # contain no 0, choose another side and shift 0 offset
                result.append(-new_shift)
                shift -= new_shift
        return result


# Time:  O(n * 2^n), len(sums) = 2^n
# Space: O(2^n)
import collections


# optimized from solution4 (not using OrderedDict), runtime: 1024 ms
class Solution4(object):
    def recoverArray(self, n, sums):
        """
        :type n: int
        :type sums: List[int]
        :rtype: List[int]
        """
        dp = {k: v for k, v in collections.Counter(sums).iteritems()}
        sorted_sums = sorted(dp.iterkeys())  # Time: O(2^n * log(2^n)) = O(n * 2^n)
        shift = 0
        result = []
        for _ in xrange(n):  # log(2^n) times, each time costs O(2^(n-len(result))), Total Time: O(2^n)
            new_dp = {}
            new_sorted_sums = []
            new_shift = sorted_sums[0]-sorted_sums[1] if dp[sorted_sums[0]] == 1 else 0
            assert(new_shift <= 0)
            for x in sorted_sums:
                if not dp[x]:
                    continue
                dp[x-new_shift] -= dp[x] if new_shift else dp[x]//2
                new_dp[x-new_shift] = dp[x]
                new_sorted_sums.append(x-new_shift)
            dp = new_dp
            sorted_sums = new_sorted_sums
            if shift in dp:  # contain 0, choose this side
                result.append(new_shift)
            else:  # contain no 0, choose another side and shift 0 offset
                result.append(-new_shift)
                shift -= new_shift
        return result


# Time:  O(n * 2^n), len(sums) = 2^n
# Space: O(2^n)
import collections


# runtime: 1720 ms
class Solution5(object):
    def recoverArray(self, n, sums):
        """
        :type n: int
        :type sums: List[int]
        :rtype: List[int]
        """
        dp = OrderedDict(sorted(collections.Counter(sums).iteritems()))  # Time: O(2^n * log(2^n)) = O(n * 2^n)
        shift = 0
        result = []
        for _ in xrange(n):  # log(2^n) times, each time costs O(2^(n-len(result))), Total Time: O(2^n)
            new_dp = OrderedDict()
            it = iter(dp)
            min_sum = next(it)
            new_shift = min_sum-next(it) if dp[min_sum] == 1 else 0
            assert(new_shift <= 0)
            for x in dp.iterkeys():
                if not dp[x]:
                    continue
                dp[x-new_shift] -= dp[x] if new_shift else dp[x]//2
                new_dp[x-new_shift] = dp[x]
            dp = new_dp
            if shift in dp:  # contain 0, choose this side
                result.append(new_shift)
            else:  # contain no 0, choose another side and shift 0 offset
                result.append(-new_shift)
                shift -= new_shift
        return result

# 1987 Hard 1987 Number of Unique Good Subsequences

In [ ]:
# Time:  O(n)
# Space: O(1)

class Solution(object):
    def numberOfUniqueGoodSubsequences(self, binary):
        """
        :type binary: str
        :rtype: int
        """
        MOD = 10**9+7
        ends0, ends1 = 0, 0
        has_zero = False
        for b in binary:
            if b == '1':
                ends1 = (ends0+ends1+1)%MOD  # curr subsequences end with 1 is all prev distinct subsequences appended by 1 and plus "1"
            else:
                ends0 = (ends0+ends1)%MOD  # curr subsequences end with 0 is all prev distinct subsequences appended by 0 and don't plus "0"
                has_zero = True
        return (ends0+ends1+int(has_zero))%MOD  # add "0" if has_zero

# 1994 Hard 1994 The Number of Good Subsets

In [ ]:
# Time:  O(n * 2^p), p is the number of primes in [1, n]
# Space: O(2^p)

import collections


class Solution(object):
    def numberOfGoodSubsets(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        def sieve_of_eratosthenes(n):  # Time: O(n * log(logn)), Space: O(n)
            if n < 2:
                return []
            primes = [2]
            is_prime = [True]*((n+1)//2)
            for i in xrange(1, len(is_prime)):
                if not is_prime[i]:
                    continue
                primes.append(2*i+1)
                for j in xrange(2*i*(i+1), len(is_prime), (2*i+1)):
                    is_prime[j] = False
            return primes

        def to_mask(primes, x):
            mask, basis = 0, 1
            for p in primes:
                if x%p == 0:
                    mask |= basis
                basis <<= 1
            return mask

        MOD = 10**9+7
        primes = sieve_of_eratosthenes(max(nums))
        dp = [0]*(1<<len(primes))  # dp[i] = the number of different good subsets of which the total product equals to the product of the primes in bitset i
        dp[0] = 1
        cnts = collections.Counter(nums)
        for x, cnt in cnts.iteritems():
            if x == 1 or any(x%(p*p) == 0 for p in primes if p*p <= x):
                continue
            mask = to_mask(primes, x)
            for i in xrange(len(dp)-1):
                if i&mask:
                    continue
                dp[i|mask] = (dp[i|mask]+cnt*dp[i])%MOD
        return (pow(2, cnts[1], MOD))*(reduce(lambda total, x: (total+x)%MOD, dp, 0)-1)%MOD

# 1998 Hard 1998 GCD Sort of an Array

In [ ]:
# Time:  O(nlogn + n * Î±(n) + m * log(logm)) ~= O(nlogn + m), m is the max of nums
# Space: O(n + m)

import itertools


class UnionFind(object):  # Time: O(n * Î±(n)), Space: O(n)
    def __init__(self, n):
        self.set = range(n)
        self.rank = [0]*n

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y):
        x_root, y_root = map(self.find_set, (x, y))
        if x_root == y_root:
            return False
        if self.rank[x_root] < self.rank[y_root]:  # union by rank
            self.set[x_root] = y_root
        elif self.rank[x_root] > self.rank[y_root]:
            self.set[y_root] = x_root
        else:
            self.set[y_root] = x_root
            self.rank[x_root] += 1
        return True


class Solution(object):
    def gcdSort(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        def modified_sieve_of_eratosthenes(n, lookup, uf):  # Time: O(n * log(logn)), Space: O(n)
            if n < 2:
                return
            is_prime = [True]*(n+1)
            for i in xrange(2, len(is_prime)):
                if not is_prime[i]:
                    continue
                for j in xrange(i+i, len(is_prime), i):
                    is_prime[j] = False
                    if j in lookup:  # modified
                        uf.union_set(i-1, j-1)

        max_num = max(nums)
        uf = UnionFind(max_num)
        modified_sieve_of_eratosthenes(max_num, set(nums), uf)
        return all(uf.find_set(a-1) == uf.find_set(b-1) for a, b in itertools.izip(nums, sorted(nums)))

# 2003 Hard 2003 Smallest Missing Genetic Value in Each Subtree

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def smallestMissingValueSubtree(self, parents, nums):
        """
        :type parents: List[int]
        :type nums: List[int]
        :rtype: List[int]
        """
        def iter_dfs(adj, nums, i, lookup):
            stk = [i]
            while stk:
                i = stk.pop()
                if nums[i] in lookup:
                    continue
                lookup.add(nums[i])
                for j in adj[i]:
                    stk.append(j)

        result = [1]*len(parents)
        i = next((i for i in xrange(len(nums)) if nums[i] == 1), -1)
        if i == -1:
            return result
        adj = [[] for _ in xrange(len(parents))]
        for j in xrange(1, len(parents)):
            adj[parents[j]].append(j)
        lookup = set()
        miss = 1
        while i >= 0:
            iter_dfs(adj, nums, i, lookup)
            while miss in lookup:
                miss += 1
            result[i] = miss
            i = parents[i]
        return result

# 2005 Hard 2005 Subtree Removal Game with Fibonacci Tree

In [ ]:
# Time:  O(n)
# Space: O(1)

class Solution(object):
    def findGameWinner(self, n):
        """
        :type n: int
        :rtype: bool
        """ 
        # a pattern appears every 6 grundy numbers in binary forms:
        # 0000,       (0000)01,       (0000)11,                 ((0000)^(0000+1))10,       (0000)11,       (0000)11
        # 0000,     (0000+1)01,     (0000+1)11,           ((0000+1)^((0000+1)+1))10,     (0000+1)11,     (0000+1)11
        # 0000, ((0000+1)+1)01, ((0000+1)+1)11,   (((0000+1)+1)^(((0000+1)+1)+1))10, ((0000+1)+1)11, ((0000+1)+1)11
        # ...
        # 0000,       (XXXX)01,       (XXXX)11,                 ((XXXX)^(XXXX+1))10,       (XXXX)11,       (XXXX)11
        # 0000,     (XXXX+1)01,     (XXXX+1)11,           ((XXXX+1)^((XXXX+1)+1))10,     (XXXX+1)11,     (XXXX+1)11
        # => grundy[6k+1] = 0
        #    grundy[6k+2] = 4k+1
        #    grundy[6k+3] = 4k+3
        #    grundy[6k+4] = 4(k^(k+1))+2
        #    grundy[6k+5] = 4k+3
        #    grundy[6k+6] = 4k+3
        return n%6 != 1


# Time:  O(n)
# Space: O(1)
class Solution2(object):
    def findGameWinner(self, n):
        """
        :type n: int
        :rtype: bool
        """ 
        grundy = [0, 1]  # 0-indexed
        for i in xrange(2, n):
            grundy[i%2] = (grundy[(i-1)%2]+1)^(grundy[(i-2)%2]+1)  # colon principle, replace the branches by a non-branching stalk of length equal to their nim sum
        return grundy[(n-1)%2] > 0

# 2009 Hard 2009 Minimum Number of Operations to Make Array Continuous

In [ ]:
# Time:  O(nlogn)
# Space: O(1)

class Solution(object):
    def minOperations(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        def unique(nums):
            left = 0
            for right in xrange(1, len(nums)):
                if nums[left] != nums[right]:
                    left += 1
                    nums[left] = nums[right]
            return left

        def erase(nums, i):
            while len(nums) > i+1:
                nums.pop()

        n = len(nums)
        nums.sort()
        erase(nums, unique(nums))
        result = l = 0
        for i in xrange(len(nums)):
            if nums[i] <= nums[i-l]+n-1:
                l += 1
        return n-l


# Time:  O(nlogn)
# Space: O(n)
class Solution2(object):
    def minOperations(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        n = len(nums)
        nums = sorted(set(nums))
        result = right = 0
        for left in xrange(len(nums)):
            while right < len(nums) and nums[right] <= nums[left]+n-1:
                right += 1
            result = max(result, right-left)
        return n-result

# 2014 Hard 2014 Longest Subsequence Repeated k Times

In [ ]:
# Time:  O(n * (n/k)!)
# Space: O(n)

import collections


class Solution(object):
    def longestSubsequenceRepeatedK(self, s, k):
        """
        :type s: str
        :type k: int
        :rtype: str
        """
        def check(s, k, curr):
            if not curr:
                return True
            i = 0
            for c in s:
                if c != curr[i]:
                    continue
                i += 1
                if i != len(curr):
                    continue
                i = 0
                k -= 1
                if not k:
                    return True
            return False

        def backtracking(s, k, curr, cnts, result):
            if not check(s, k, curr):
                return
            if len(curr) > len(result):
                result[:] = curr
            for c in reversed(string.ascii_lowercase):
                if cnts[c] < k:
                    continue
                cnts[c] -= k
                curr.append(c)
                backtracking(s, k, curr, cnts, result)
                curr.pop()
                cnts[c] += k
                    
        cnts = collections.Counter(s)
        new_s = []
        for c in s:
            if cnts[c] < k:
                continue
            new_s.append(c)
        result =[]
        backtracking(new_s, k, [], cnts, result)
        return "".join(result)

# 2019 Hard 2019 The Score of Students Solving Math Expression

In [ ]:
# Time:  O(n^3 * a^2)
# Space: O(n^2)

class Solution(object):
    def scoreOfStudents(self, s, answers):
        """
        :type s: str
        :type answers: List[int]
        :rtype: int
        """
        MAX_ANS = 1000
        n = (len(s)+1)//2
        dp = [[set() for _ in xrange(n)] for _ in xrange(n)]
        for i in xrange(n):
            dp[i][i].add(int(s[i*2]))
        for l in xrange(1, n):
            for left in xrange(n-l):
                right = left+l
                for k in xrange(left, right):
                    if s[2*k+1] == '+':
                        dp[left][right].update((x+y for x in dp[left][k] for y in dp[k+1][right] if x+y <= MAX_ANS))
                    else:
                        dp[left][right].update((x*y for x in dp[left][k] for y in dp[k+1][right] if x*y <= MAX_ANS))
        target = eval(s)
        return sum(5 if ans == target else 2 if ans in dp[0][-1] else 0 for ans in answers)


# Time:  O(n^3 * a^2)
# Space: O(n^2)
class Solution2(object):
    def scoreOfStudents(self, s, answers):
        """
        :type s: str
        :type answers: List[int]
        :rtype: int
        """
        MAX_ANS = 1000
        def evaluate(s):
            def compute(operands, operators):
                right, left = operands.pop(), operands.pop()
                operands.append(ops[operators.pop()](left, right))

            ops = {'+':operator.add, '*':operator.mul}
            precedence = {'+':0, '*':1}
            operands, operators, operand = [], [], 0
            for c in s:
                if c.isdigit():
                    operands.append(int(c))
                else:
                    while operators and precedence[operators[-1]] >= precedence[c]:
                        compute(operands, operators)
                    operators.append(c)
            while operators:
                compute(operands, operators)
            return operands[-1]

        n = (len(s)+1)//2
        dp = [[set() for _ in xrange(n)] for _ in xrange(n)]
        for i in xrange(n):
            dp[i][i].add(int(s[i*2]))
        for l in xrange(1, n):
            for left in xrange(n-l):
                right = left+l
                for k in xrange(left, right):
                    if s[2*k+1] == '+':
                        dp[left][right].update((x+y for x in dp[left][k] for y in dp[k+1][right] if x+y <= MAX_ANS))
                    else:
                        dp[left][right].update((x*y for x in dp[left][k] for y in dp[k+1][right] if x*y <= MAX_ANS))
        target = evaluate(s)
        return sum(5 if ans == target else 2 if ans in dp[0][-1] else 0 for ans in answers)

# 2025 Hard 2025 Maximum Number of Ways to Partition an Array

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


class Solution(object):
    def waysToPartition(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        total = sum(nums)
        right = collections.Counter()
        prefix = 0
        for i in xrange(len(nums)-1):
            prefix += nums[i]
            right[prefix-(total-prefix)] += 1
        result = right[0]
        left = collections.Counter()
        prefix = 0
        for x in nums:
            result = max(result, left[k-x]+right[-(k-x)])
            prefix += x
            left[prefix-(total-prefix)] += 1
            right[prefix-(total-prefix)] -= 1
        return result

# 2035 Hard 2035 Partition Array Into Two Arrays to Minimize Sum Difference

In [ ]:
# Time:  O(n * 2^n)
# Space: O(2^n)

import itertools
import bisect


class Solution(object):
    def minimumDifference(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        left, right = nums[:len(nums)//2], nums[len(nums)//2:]
        total1, total2 = sum(left), sum(right)
        result = float("inf")
        for k in xrange(len(left)+1): 
            sums = sorted(2*sum(comb)-total1 for comb in itertools.combinations(left, k))
            for comb in itertools.combinations(right, len(left)-k): 
                diff = 2*sum(comb)-total2
                i = bisect.bisect_left(sums, -diff)
                if i < len(sums):
                    result = min(result, abs(sums[i]+diff))
                if i > 0:
                    result = min(result, abs(sums[i-1]+diff))
        return result

# 2040 Hard 2040 Kth Smallest Product of Two Sorted Arrays

In [ ]:
# Time:  O((m + n) * logr), r is the range size of [min(products), max(products)]
# Space: O(1)

class Solution(object):
    def kthSmallestProduct(self, nums1, nums2, k):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :type k: int
        :rtype: int
        """
        def check(nums1, nums2, k, neg_cnt, target):
            cnt = 0
            left, right = 0, len(nums2)-1
            direction = reversed if target >= 0 else lambda x: x
            for i in direction(xrange(neg_cnt)):
                while left < len(nums2) and nums1[i]*nums2[left] > target:
                    left += 1
                cnt += (len(nums2)-1)-left+1
            direction = (lambda x: x) if target >= 0 else reversed
            for i in direction(xrange(neg_cnt, len(nums1))): 
                if nums1[i] == 0: 
                    if target >= 0:
                        cnt += len(nums2)
                    continue
                while right >= 0 and nums1[i]*nums2[right] > target:
                    right -= 1
                cnt += right-0+1
            return cnt >= k

        neg_cnt = sum(x < 0 for x in nums1)
        left = min(nums1[i]*nums2[j] for i in (0, -1) for j in (0, -1))
        right = max(nums1[i]*nums2[j] for i in (0, -1) for j in (0, -1))
        while left <= right:
            mid = left + (right-left)//2
            if check(nums1, nums2, k, neg_cnt, mid):
                right = mid-1
            else:
                left = mid+1
        return left

# 2045 Hard 2045 Second Minimum Time to Reach Destination

In [ ]:
# Time:  O(|V| + |E|) = O(|E|) since graph is connected, O(|E|) >= O(|V|) 
# Space: O(|V| + |E|) = O(|E|)

class Solution(object):
    def secondMinimum(self, n, edges, time, change):
        """
        :type n: int
        :type edges: List[List[int]]
        :type time: int
        :type change: int
        :rtype: int
        """
        # Template:
        # https://github.com/kamyu104/LeetCode-Solutions/blob/master/Python/find-if-path-exists-in-graph.py
        def bi_bfs(adj, start, target):
            left, right = {start}, {target}
            lookup = set()
            result = steps = 0
            while left and (not result or result+2 > steps):  # modified
                for u in left:
                    lookup.add(u)
                new_left = set()
                for u in left: 
                    if u in right:
                        if not result:  # modified
                            result = steps
                        elif result < steps:  # modifeid
                            return result+1
                    for v in adj[u]:
                        if v in lookup:
                            continue
                        new_left.add(v)
                left = new_left
                steps += 1
                if len(left) > len(right): 
                    left, right = right, left
            return result+2  # modified

        def calc_time(time, change, dist):
            result = 0
            for _ in xrange(dist):
                if result//change%2:
                    result = (result//change+1)*change
                result += time
            return result

        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u-1].append(v-1)
            adj[v-1].append(u-1)
        return calc_time(time, change, bi_bfs(adj, 0, n-1))


# Time:  O(|V| + |E|) = O(|E|) since graph is connected, O(|E|) >= O(|V|) 
# Space: O(|V| + |E|) = O(|E|)
class Solution2(object):
    def secondMinimum(self, n, edges, time, change):
        """
        :type n: int
        :type edges: List[List[int]]
        :type time: int
        :type change: int
        :rtype: int
        """
        INF = float("inf")
        def bfs(adj, start):
            q = [start]
            dist = [INF]*len(adj)
            dist[start] = 0
            while q:
                new_q = []
                for u in q:
                    for v in adj[u]:
                        if dist[v] != INF:
                            continue
                        dist[v] = dist[u]+1
                        new_q.append(v)
                q = new_q
            return dist

        def calc_time(time, change, dist):
            result = 0
            for _ in xrange(dist):
                if result//change%2:
                    result = (result//change+1)*change
                result += time
            return result

        adj = [[] for _ in xrange(n)]
        for u, v in edges:
            adj[u-1].append(v-1)
            adj[v-1].append(u-1)
        dist_to_end, dist_to_start = bfs(adj, 0), bfs(adj, n-1)

        dist = dist_to_end[n-1]+2  # always exists
        for i in xrange(n):  # case of detour
            if dist_to_end[i]+dist_to_start[i] == dist_to_end[n-1]:
                continue
            dist = min(dist, dist_to_end[i]+dist_to_start[i])  # find second min
            if dist == dist_to_end[n-1]+1:
                break
        return calc_time(time, change, dist)

# 2050 Hard 2050 Parallel Courses III

In [ ]:
# Time:  O(|V| + |E|)
# Space: O(|E|)

class Solution(object):
    def minimumTime(self, n, relations, time):
        """
        :type n: int
        :type relations: List[List[int]]
        :type time: List[int]
        :rtype: int
        """
        adj = [[] for _ in xrange(n)]
        in_degree = [0]*n
        for prev, nxt in relations:
            adj[prev-1].append(nxt-1)
            in_degree[nxt-1] += 1
        q = [u for u in xrange(n) if not in_degree[u]]
        dist = [time[u] if not in_degree[u] else 0 for u in xrange(n)] 
        while q:
            new_q = []
            for u in q:
                for v in adj[u]:
                    dist[v] = max(dist[v], dist[u]+time[v])
                    in_degree[v] -= 1
                    if not in_degree[v]:
                        new_q.append(v)
            q = new_q
        return max(dist)

# 2056 Hard 2056 Number of Valid Move Combinations On Chessboard

In [ ]:
# Time:  O(n^p) = O(1), n is the max number of possible moves for each piece, and n is at most 29
#                     , p is the number of pieces, and p is at most 4
# Space: O(1)

class Solution(object):
    def countCombinations(self, pieces, positions):
        """
        :type pieces: List[str]
        :type positions: List[List[int]]
        :rtype: int
        """
        directions = {"rook": [(0, 1), (1, 0), (0, -1), (-1, 0)],
                      "bishop": [(1, 1), (1, -1), (-1, 1), (-1, -1)],
                      "queen" : [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]}
        all_mask = 2**7-1  # at most 7 seconds in 8x8 board
        def backtracking(pieces, positions, i, lookup):
            if i == len(pieces):
                return 1
            result = 0
            r, c = positions[i]
            r, c = r-1, c-1
            mask = all_mask
            if not (lookup[r][c]&mask):
                lookup[r][c] += mask  # stopped at (r, c)
                result += backtracking(pieces, positions, i+1, lookup)
                lookup[r][c] -= mask          
            for dr, dc in directions[pieces[i]]:
                bit, nr, nc = 1, r+dr, c+dc
                mask = all_mask  # (mask&bit == 1): (log2(bit)+1)th second is occupied
                while 0 <= nr < 8 and 0 <= nc < 8 and not (lookup[nr][nc]&bit):
                    lookup[nr][nc] += bit
                    mask -= bit
                    if not (lookup[nr][nc]&mask):  # stopped at (nr, nc)
                        lookup[nr][nc] += mask
                        result += backtracking(pieces, positions, i+1, lookup)
                        lookup[nr][nc] -= mask
                    bit, nr, nc = bit<<1, nr+dr, nc+dc
                while bit>>1:
                    bit, nr, nc = bit>>1, nr-dr, nc-dc
                    lookup[nr][nc] -= bit
            return result

        return backtracking(pieces, positions, 0, [[0]*8 for _ in range(8)])

# 2060 Hard 2060 Check if an Original String Exists Given Two Encoded Strings

In [ ]:
# Time:  O(m * n * k), k is the max number of consecutive digits in s1 and s2
# Space: O(m * n * k)

# top-down dp (faster since accessing less states)
class Solution(object):
    def possiblyEquals(self, s1, s2):
        """
        :type s1: str
        :type s2: str
        :rtype: bool
        """
        def general_possible_numbers(s):  # Time: O(2^l), Space: O(2^l), l is the length of consecutive digits, and l is at most 3
            dp = [set() for _ in xrange(len(s))]
            for i in xrange(len(s)):
                curr, basis = 0, 1
                for j in reversed(xrange(i+1)):
                    curr += int(s[j])*basis
                    basis *= 10
                    if s[j] == '0':
                        continue
                    if j == 0:
                        dp[i].add(curr)
                    else:
                        dp[i].update(x+curr for x in dp[j-1])        
            return dp[-1]

        def optimized_possible_numbers(s):
            assert(len(s) <= 3)
            result = {int(s)}
            if len(s) >= 2:
                if s[1] != '0':
                    result.add(int(s[:1])+int(s[1:]))
            if len(s) >= 3:
                if s[2] != '0':
                    result.add(int(s[:2])+int(s[2:]))
                    if s[1] != '0':
                        result.add(int(s[0:1])+int(s[1:2])+int(s[2:]))
            return result
    
        def memoization(s1, s2, i, j, k, lookup):
            if (i, j, k) not in lookup:
                if i == len(s1) and j == len(s2):
                    lookup[(i, j, k)] = (k == 0)
                elif i != len(s1) and s1[i].isdigit():
                    lookup[(i, j, k)] = False
                    for ni in xrange(i+1, len(s1)+1):
                        if ni == len(s1) or not s1[ni].isdigit():
                            break
                    for x in optimized_possible_numbers(s1[i:ni]):
                        if memoization(s1, s2, ni, j, k+x, lookup):
                            lookup[(i, j, k)] = True
                            break
                elif j != len(s2) and s2[j].isdigit():
                    lookup[(i, j, k)] = False
                    for nj in xrange(j+1, len(s2)+1):
                        if nj == len(s2) or not s2[nj].isdigit():
                            break
                    for x in optimized_possible_numbers(s2[j:nj]):
                        if memoization(s1, s2, i, nj, k-x, lookup):
                            lookup[(i, j, k)] = True
                            break
                elif k < 0:
                    lookup[(i, j, k)] = memoization(s1, s2, i+1, j, k+1, lookup) if i != len(s1) else False
                elif k > 0:
                    lookup[(i, j, k)] = memoization(s1, s2, i, j+1, k-1, lookup) if j != len(s2) else False
                else:
                    lookup[(i, j, k)] = memoization(s1, s2, i+1, j+1, k, lookup) if i != len(s1) and j != len(s2) and s1[i] == s2[j] else False
            return lookup[(i, j, k)]

        return memoization(s1, s2, 0, 0, 0, {})


# Time:  O(m * n * k), k is the max number of consecutive digits in s1 and s2
# Space: O(m * n * k)
# top-down dp (faster since accessing less states)
class Solution2(object):
    def possiblyEquals(self, s1, s2):
        """
        :type s1: str
        :type s2: str
        :rtype: bool
        """
        def memoization(s1, s2, i, j, k, lookup):
            if (i, j, k) not in lookup:
                if i == len(s1) and j == len(s2):
                    lookup[(i, j, k)] = (k == 0)
                elif i != len(s1) and s1[i].isdigit():
                    lookup[(i, j, k)] = False
                    for ni in xrange(i+1, len(s1)+1):
                        if (ni == len(s1) or s1[ni] != '0') and memoization(s1, s2, ni, j, k+int(s1[i:ni]), lookup):
                            lookup[(i, j, k)] = True
                            break
                        if ni == len(s1) or not s1[ni].isdigit():
                            break
                elif j != len(s2) and s2[j].isdigit():
                    lookup[(i, j, k)] = False
                    for nj in xrange(j+1, len(s2)+1):
                        if (nj == len(s2) or s2[nj] != '0') and memoization(s1, s2, i, nj, k-int(s2[j:nj]), lookup):
                            lookup[(i, j, k)] = True
                            break
                        if nj == len(s2) or not s2[nj].isdigit():
                            break
                elif k < 0:
                    lookup[(i, j, k)] = memoization(s1, s2, i+1, j, k+1, lookup) if i != len(s1) else False
                elif k > 0:
                    lookup[(i, j, k)] = memoization(s1, s2, i, j+1, k-1, lookup) if j != len(s2) else False
                else:
                    lookup[(i, j, k)] = memoization(s1, s2, i+1, j+1, k, lookup) if i != len(s1) and j != len(s2) and s1[i] == s2[j] else False
            return lookup[(i, j, k)]

        return memoization(s1, s2, 0, 0, 0, {})


# Time:  O(m * n * k), k is the max number of consecutive digits in s1 and s2
# Space: O(min(m, n) * k)
# bottom-up dp
class Solution3(object):
    def possiblyEquals(self, s1, s2):
        """
        :type s1: str
        :type s2: str
        :rtype: bool
        """
        MAX_DIGIT_LEN = 3
        w = 1+MAX_DIGIT_LEN
        dp = [[set() for _ in xrange(len(s2)+1)] for _ in xrange(w)]
        dp[0][0].add(0)
        for i in xrange(len(s1)+1):
            if i:
                dp[(i-1)%w] = [set() for _ in xrange(len(s2)+1)]
            if i != len(s1) and s1[i] == '0':
                continue
            for j in xrange(len(s2)+1):
                for k in dp[i%w][j]:
                    if i != len(s1) and j != len(s2) and s1[i] == s2[j] and k == 0:
                        dp[(i+1)%w][j+1].add(k)
                    if k <= 0 and i != len(s1):
                        if not s1[i].isdigit():
                            if k:
                                dp[(i+1)%w][j].add(k+1)
                        elif s1[i] != '0':
                            curr = 0
                            for ni in xrange(i, len(s1)):
                                if not s1[ni].isdigit():
                                    break
                                curr = curr*10 + int(s1[ni])
                                dp[(ni+1)%w][j].add(k+curr)
                    if k >= 0 and j != len(s2):
                        if not s2[j].isdigit():
                            if k:
                                dp[i%w][j+1].add(k-1)
                        elif s2[j] != '0':
                            curr = 0
                            for nj in xrange(j, len(s2)):
                                if not s2[nj].isdigit():
                                    break
                                curr = curr*10 + int(s2[nj])
                                dp[i%w][nj+1].add(k-curr)
        return 0 in dp[len(s1)%w][len(s2)]

# 2065 Hard 2065 Maximum Path Quality of a Graph

In [ ]:
# Time: O(|V| + |E| + 4^(maxTime/min(times))) = O(|V| + |E| + 4^10)
# Time: O(|V| + |E|)

class Solution(object):
    def maximalPathQuality(self, values, edges, maxTime):
        """
        :type values: List[int]
        :type edges: List[List[int]]
        :type maxTime: int
        :rtype: int
        """
        def iter_dfs(values, adj, maxTime):
            lookup, lookup2 = [0]*len(adj), set()
            result = 0
            stk = [(1, (0, maxTime, 0))]
            while stk:
                step, args = stk.pop()
                if step == 1:
                    u, time, total = args
                    lookup[u] += 1
                    if lookup[u] == 1:
                        total += values[u]
                    if not u:
                        result = max(result, total)
                    stk.append((4, (u,)))
                    for v, t in reversed(adj[u]):
                        if (u, v) in lookup2 or time < t:  # same directed edge won't be visited twice
                            continue
                        stk.append((3, (u, v)))
                        stk.append((1, (v, time-t, total)))
                        stk.append((2, (u, v)))
                elif step == 2:
                    u, v = args
                    lookup2.add((u, v))
                elif step == 3:
                    u, v = args
                    lookup2.remove((u, v))
                elif step == 4:
                    u = args[0]
                    lookup[u] -= 1
            return result

        adj = [[] for _ in xrange(len(values))]
        for u, v, t in edges:
            adj[u].append((v, t))
            adj[v].append((u, t))
        return iter_dfs(values, adj, maxTime)


# Time: O(|V| + |E| + 4^(maxTime/min(times))) = O(|V| + |E| + 4^10)
# Time: O(|V| + |E|)
class Solution2(object):
    def maximalPathQuality(self, values, edges, maxTime):
        """
        :type values: List[int]
        :type edges: List[List[int]]
        :type maxTime: int
        :rtype: int
        """
        def dfs(values, adj, u, time, total, lookup, lookup2, result):
            lookup[u] += 1
            if lookup[u] == 1:
                total += values[u]
            if not u:
                result[0] = max(result[0], total)
            for v, t in adj[u]:
                if (u, v) in lookup2 or time < t:  # same directed edge won't be visited twice
                    continue
                lookup2.add((u, v))
                dfs(values, adj, v, time-t, total, lookup, lookup2, result)
                lookup2.remove((u, v))
            lookup[u] -= 1

        adj = [[] for _ in xrange(len(values))]
        for u, v, t in edges:
            adj[u].append((v, t))
            adj[v].append((u, t))
        result = [0]
        dfs(values, adj, 0, maxTime, 0, [0]*len(adj), set(), result)
        return result[0]


# Time: O(|V| + |E| + 4^(maxTime/min(times))) = O(|V| + |E| + 4^10)
# Time: O(|V| + |E|)
class Solution3(object):
    def maximalPathQuality(self, values, edges, maxTime):
        """
        :type values: List[int]
        :type edges: List[List[int]]
        :type maxTime: int
        :rtype: int
        """
        def dfs(values, adj, u, time, total, lookup, lookup2):
            lookup[u] += 1
            if lookup[u] == 1:
                total += values[u]
            result = total if not u else 0
            for v, t in adj[u]:
                if (u, v) in lookup2 or time < t:  # same directed edge won't be visited twice
                    continue
                lookup2.add((u, v))
                result = max(result, dfs(values, adj, v, time-t, total, lookup, lookup2))
                lookup2.remove((u, v))
            lookup[u] -= 1
            return result

        adj = [[] for _ in xrange(len(values))]
        for u, v, t in edges:
            adj[u].append((v, t))
            adj[v].append((u, t))
        return dfs(values, adj, 0, maxTime, 0, [0]*len(adj), set())

# 2071 Hard 2071 Maximum Number of Tasks You Can Assign

In [ ]:
# Time:  O(n * (logn)^2)
# Space: O(n)

from sortedcontainers import SortedList


class Solution(object):
    def maxTaskAssign(self, tasks, workers, pills, strength):
        """
        :type tasks: List[int]
        :type workers: List[int]
        :type pills: int
        :type strength: int
        :rtype: int
        """
        def check(tasks, workers, pills, strength, x):
            w = SortedList(workers[-x:])
            for task in tasks[-x:]:  # greedily assign
                i = w.bisect_left(task)
                if i != len(w):
                    w.pop(i)
                    continue
                if pills:
                    i = w.bisect_left(task-strength)
                    if i != len(w):
                        w.pop(i)
                        pills -= 1
                        continue
                return False
            return True

        tasks.sort(reverse=True)
        workers.sort()
        left, right = 1, min(len(workers), len(tasks))
        while left <= right:
            mid = left + (right-left)//2
            if not check(tasks, workers, pills, strength, mid):
                right = mid-1
            else:
                left = mid+1
        return right


# Time:  O(n^2 * logn)
# Space: O(n)
class Solution2(object):
    def maxTaskAssign(self, tasks, workers, pills, strength):
        """
        :type tasks: List[int]
        :type workers: List[int]
        :type pills: int
        :type strength: int
        :rtype: int
        """
        def check(tasks, workers, pills, strength, x):
            w = workers[-x:]
            for task in tasks[-x:]:  # greedily assign
                i = bisect.bisect_left(w, task)
                if i != len(w):
                    w.pop(i)
                    continue
                if pills:
                    i = bisect.bisect_left(w, task-strength)
                    if i != len(w):
                        w.pop(i)
                        pills -= 1
                        continue
                return False
            return True

        tasks.sort(reverse=True)
        workers.sort()
        left, right = 1, min(len(workers), len(tasks))
        while left <= right:
            mid = left + (right-left)//2
            if not check(tasks, workers, pills, strength, mid):
                right = mid-1
            else:
                left = mid+1
        return right

# 2076 Hard 2076 Process Restricted Friend Requests

In [ ]:
# Time:  O(n * (alpha(n) + r)) = O(n * r)
# Space: O(n)

class UnionFind(object):  # Time: O(n * alpha(n)), Space: O(n)
    def __init__(self, n):
        self.set = range(n)
        self.rank = [0]*n

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y):
        x, y = self.find_set(x), self.find_set(y)
        if x == y:
            return False
        if self.rank[x] > self.rank[y]:  # union by rank
            x, y = y, x
        self.set[x] = self.set[y]
        if self.rank[x] == self.rank[y]:
            self.rank[y] += 1
        return True


class Solution(object):
    def friendRequests(self, n, restrictions, requests):
        """
        :type n: int
        :type restrictions: List[List[int]]
        :type requests: List[List[int]]
        :rtype: List[bool]
        """
        result = []
        uf = UnionFind(n)
        for u, v in requests:
            pu, pv = uf.find_set(u), uf.find_set(v)
            ok = True
            for x, y in restrictions:
                px, py = uf.find_set(x), uf.find_set(y)
                if {px, py} == {pu, pv}:
                    ok = False
                    break
            result.append(ok)
            if ok:
                uf.union_set(u, v) 
        return result

# 2088 Hard 2088 Count Fertile Pyramids in a Land

In [ ]:
# Time:  O(m * n)
# Space: O(n)

class Solution(object):
    def countPyramids(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        def count(grid, reverse):
            def get_grid(i, j):
                return grid[~i][j] if reverse else grid[i][j]

            result = 0
            dp = [0]*len(grid[0])
            for i in xrange(1, len(grid)):
                new_dp = [0]*len(grid[0])
                for j in xrange(1, len(grid[0])-1):
                    if get_grid(i, j) == get_grid(i-1, j-1) == get_grid(i-1, j) == get_grid(i-1, j+1) == 1:
                        new_dp[j] = min(dp[j-1], dp[j+1])+1
                dp = new_dp
                result += sum(dp)
            return result
        
        return count(grid, False) + count(grid, True)


# Time:  O(m * n)
# Space: O(m * n)
class Solution2(object):
    def countPyramids(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        def count(grid):
            dp = [[0 for _ in xrange(len(grid[0]))] for _ in xrange(len(grid))]
            for i in xrange(1, len(grid)):
                for j in xrange(1, len(grid[0])-1):
                    if grid[i][j] == grid[i-1][j-1] == grid[i-1][j] == grid[i-1][j+1] == 1:
                        dp[i][j] = min(dp[i-1][j-1], dp[i-1][j], dp[i-1][j+1])+1
            return sum(sum(row) for row in dp)
        
        return count(grid) + count(grid[::-1])

# 2092 Hard 2092 Find All People With Secret

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import collections


class Solution(object):
    def findAllPeople(self, n, meetings, firstPerson):
        """
        :type n: int
        :type meetings: List[List[int]]
        :type firstPerson: int
        :rtype: List[int]
        """
        meetings.sort(key=lambda x: x[2])
        result = {0, firstPerson}
        adj = collections.defaultdict(list)
        for i, (x, y, _) in enumerate(meetings):
            adj[x].append(y)
            adj[y].append(x)
            if i+1 != len(meetings) and meetings[i+1][2] == meetings[i][2]:
                continue
            q = [i for i in adj.iterkeys() if i in result]
            while q:
                new_q = []
                for u in q:
                    for v in adj[u]:
                        if v in result:
                            continue
                        result.add(v)
                        new_q.append(v)
                q = new_q
            adj = collections.defaultdict(list)
        return list(result)


# Time:  O(nlogn)
# Space: O(n)
import collections


class Solution2(object):
    def findAllPeople(self, n, meetings, firstPerson):
        """
        :type n: int
        :type meetings: List[List[int]]
        :type firstPerson: int
        :rtype: List[int]
        """
        meetings.sort(key=lambda x: x[2])
        result = {0, firstPerson}
        adj = collections.defaultdict(list)
        for i, (x, y, _) in enumerate(meetings):
            adj[x].append(y)
            adj[y].append(x)
            if i+1 != len(meetings) and meetings[i+1][2] == meetings[i][2]:
                continue
            stk = [i for i in adj.iterkeys() if i in result]
            while stk:
                u = stk.pop()
                for v in adj[u]:
                    if v in result:
                        continue
                    result.add(v)
                    stk.append(v)
            adj = collections.defaultdict(list)
        return list(result)


# Time:  O(nlogn)
# Space: O(n)
class UnionFind(object):  # Time: O(n * alpha(n)), Space: O(n)
    def __init__(self, n):
        self.set = range(n)
        self.rank = [0]*n

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y):
        x, y = self.find_set(x), self.find_set(y)
        if x == y:
            return False
        if self.rank[x] > self.rank[y]:  # union by rank
            x, y = y, x
        self.set[x] = self.set[y]
        if self.rank[x] == self.rank[y]:
            self.rank[y] += 1
        return True

    def reset(self, x):
        self.set[x] = x
        self.rank[x] = 0


class Solution3(object):
    def findAllPeople(self, n, meetings, firstPerson):
        """
        :type n: int
        :type meetings: List[List[int]]
        :type firstPerson: int
        :rtype: List[int]
        """
        meetings.sort(key=lambda x: x[2])
        uf = UnionFind(n)
        uf.union_set(0, firstPerson)
        group = set()
        for i, (x, y, _) in enumerate(meetings):
            group.add(x)
            group.add(y)
            uf.union_set(x, y)
            if i+1 != len(meetings) and meetings[i+1][2] == meetings[i][2]:
                continue
            while group:
                x = group.pop()
                if uf.find_set(x) != uf.find_set(0):
                    uf.reset(x)
        return [i for i in xrange(n) if uf.find_set(i) == uf.find_set(0)]

# 2097 Hard 2097 Valid Arrangement of Pairs

In [ ]:
# Time:  O(|V| + |E|)
# Space: O(|V| + |E|)

import collections


# Hierholzer Algorithm
class Solution(object):
    def validArrangement(self, pairs):
        """
        :type pairs: List[List[int]]
        :rtype: List[List[int]]
        """
        adj = collections.defaultdict(list)
        degree = collections.defaultdict(int)
        for u, v in pairs: 
            adj[u].append(v)
            degree[u] += 1
            degree[v] -= 1       
        result = []
        stk = [next((u for u, c in degree.iteritems() if c == 1), next(degree.iterkeys()))]
        while stk:
            while adj[stk[-1]]: 
                stk.append(adj[stk[-1]].pop())
            result.append(stk.pop())
        result.reverse()
        return [[result[i], result[i+1]] for i in xrange(len(result)-1)]

# 2102 Hard 2102 Sequentially Ordinal Rank Tracker

In [ ]:
# Time:  add: O(logn)
#        get: O(logn)
# Space: O(n)

from sortedcontainers import SortedList


class SORTracker(object):

    def __init__(self):
        self.__sl = SortedList()
        self.__i = 0

    def add(self, name, score):
        """
        :type name: str
        :type score: int
        :rtype: None
        """
        self.__sl.add((-score, name))
        
    def get(self):
        """
        :rtype: str
        """
        self.__i += 1
        return self.__sl[self.__i-1][1]

# 2106 Hard 2106 Maximum Fruits Harvested After at Most K Steps

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def maxTotalFruits(self, fruits, startPos, k):
        """
        :type fruits: List[List[int]]
        :type startPos: int
        :type k: int
        :rtype: int
        """
        max_pos = max(startPos, fruits[-1][0])
        cnt = [0]*(1+max_pos)
        for p, a in fruits:
            cnt[p] = a
        prefix = [0]
        for x in cnt:
            prefix.append(prefix[-1]+x)
        result = 0
        for left_dist in xrange(min(startPos, k)+1):
            right_dist = max(k-2*left_dist, 0)            
            left, right = startPos-left_dist, min(startPos+right_dist, max_pos)
            result = max(result, prefix[right+1]-prefix[left])
        for right_dist in xrange(min(max_pos-startPos, k)+1):
            left_dist = max(k-2*right_dist, 0) 
            left, right = max(startPos-left_dist, 0), startPos+right_dist
            result = max(result, prefix[right+1]-prefix[left])
        return result

# 2117 Hard 2117 Abbreviating the Product of a Range

In [ ]:
# Time:  O(r - l)
# Space: O(1)

import math


class Solution(object):
    def abbreviateProduct(self, left, right):
        """
        :type left: int
        :type right: int
        :rtype: str
        """
        PREFIX_LEN = SUFFIX_LEN = 5
        MOD = 10**(PREFIX_LEN+SUFFIX_LEN)
        curr, zeros = 1, 0
        abbr = False
        for i in xrange(left, right+1):
            curr *= i
            while not curr%10:
                curr //= 10
                zeros += 1
            q, curr = divmod(curr, MOD)
            if q:
                abbr = True
        if not abbr:
            return "%se%s" % (curr, zeros)
        decimal = reduce(lambda x, y: (x+y)%1, (math.log10(i) for i in xrange(left, right+1)))
        prefix = str(int(10**(decimal+(PREFIX_LEN-1))))
        suffix = str(curr % 10**SUFFIX_LEN).zfill(SUFFIX_LEN)
        return "%s...%se%s" % (prefix, suffix, zeros)

# 2122 Hard 2122 Recover the Original Array

In [ ]:
# Time:  O(n^2)
# Space: O(n)

import collections


class Solution(object):
    def recoverArray(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        def check(k, cnt, result):
            for x in nums:
                if cnt[x] == 0:
                    continue
                if cnt[x+2*k] == 0:
                    return False
                cnt[x] -= 1
                cnt[x+2*k] -= 1
                result.append(x+k)
            return True
            
        nums.sort()
        cnt = collections.Counter(nums)
        for i in xrange(1, len(nums)//2+1):
            k = nums[i]-nums[0]
            if k == 0 or k%2:
                continue
            k //= 2
            result = []
            if check(k, collections.Counter(cnt), result):
                return result
        return []

# 2123 Hard 2123 Minimum Operations to Remove Adjacent Ones in Matrix

In [ ]:
# Time:  O(E * sqrt(V)) = O(m * n * sqrt(m * n))
# Space: O(V) = O(m * n)

from functools import partial

# Time:  O(E * sqrt(V))
# Space: O(V)
# Source code from http://code.activestate.com/recipes/123641-hopcroft-karp-bipartite-matching/
# Hopcroft-Karp bipartite max-cardinality matching and max independent set
# David Eppstein, UC Irvine, 27 Apr 2002
def bipartiteMatch(graph):
    '''Find maximum cardinality matching of a bipartite graph (U,V,E).
    The input format is a dictionary mapping members of U to a list
    of their neighbors in V.  The output is a triple (M,A,B) where M is a
    dictionary mapping members of V to their matches in U, A is the part
    of the maximum independent set in U, and B is the part of the MIS in V.
    The same object may occur in both U and V, and is treated as two
    distinct vertices if this happens.'''
    
    # initialize greedy matching (redundant, but faster than full search)
    matching = {}
    for u in graph:
        for v in graph[u]:
            if v not in matching:
                matching[v] = u
                break
    
    while 1:
        # structure residual graph into layers
        # pred[u] gives the neighbor in the previous layer for u in U
        # preds[v] gives a list of neighbors in the previous layer for v in V
        # unmatched gives a list of unmatched vertices in final layer of V,
        # and is also used as a flag value for pred[u] when u is in the first layer
        preds = {}
        unmatched = []
        pred = dict([(u,unmatched) for u in graph])
        for v in matching:
            del pred[matching[v]]
        layer = list(pred)
        
        # repeatedly extend layering structure by another pair of layers
        while layer and not unmatched:
            newLayer = {}
            for u in layer:
                for v in graph[u]:
                    if v not in preds:
                        newLayer.setdefault(v,[]).append(u)
            layer = []
            for v in newLayer:
                preds[v] = newLayer[v]
                if v in matching:
                    layer.append(matching[v])
                    pred[matching[v]] = v
                else:
                    unmatched.append(v)
        
        # did we finish layering without finding any alternating paths?
        if not unmatched:
            unlayered = {}
            for u in graph:
                for v in graph[u]:
                    if v not in preds:
                        unlayered[v] = None
            return (matching,list(pred),list(unlayered))

        # recursively search backward through layers to find alternating paths
        # recursion returns true if found path, false otherwise
        def recurse(v):
            if v in preds:
                L = preds[v]
                del preds[v]
                for u in L:
                    if u in pred:
                        pu = pred[u]
                        del pred[u]
                        if pu is unmatched or recurse(pu):
                            matching[v] = u
                            return 1
            return 0
        
        def recurse_iter(v):
            def divide(v):
                if v not in preds:
                    return
                L = preds[v]
                del preds[v]
                for u in L :
                    if u in pred and pred[u] is unmatched:  # early return
                        del pred[u]
                        matching[v] = u
                        ret[0] = True
                        return
                stk.append(partial(conquer, v, iter(L)))

            def conquer(v, it):
                for u in it:
                    if u not in pred:
                        continue
                    pu = pred[u]
                    del pred[u]
                    stk.append(partial(postprocess, v, u, it))
                    stk.append(partial(divide, pu))
                    return

            def postprocess(v, u, it):
                if not ret[0]:
                    stk.append(partial(conquer, v, it))
                    return
                matching[v] = u

            ret, stk = [False], []
            stk.append(partial(divide, v))
            while stk:
                stk.pop()()
            return ret[0]

        for v in unmatched: recurse_iter(v)


import collections


class Solution(object):
    def minimumOperations(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        def iter_dfs(grid, i, j, lookup, adj):
            if lookup[i][j]:
                return
            lookup[i][j] = True
            stk = [(i, j, (i+j)%2)]
            while stk:
                i, j, color = stk.pop()
                for di, dj in directions:
                    ni, nj = i+di, j+dj
                    if not (0 <= ni < len(grid) and 0 <= nj < len(grid[0]) and grid[ni][nj]):
                        continue
                    if not color:
                        adj[len(grid[0])*ni+nj].append(len(grid[0])*i+j)
                    if lookup[ni][nj]:
                        continue
                    lookup[ni][nj] = True
                    stk.append((ni, nj, color^1))

        adj = collections.defaultdict(list)
        lookup = [[False]*len(grid[0]) for _ in xrange(len(grid))]
        for i in xrange(len(grid)):
            for j in xrange(len(grid[0])):
                if not grid[i][j]:
                    continue
                iter_dfs(grid, i, j, lookup, adj)
        return len(bipartiteMatch(adj)[0])

# 2127 Hard 2127 Maximum Employees to Be Invited to a Meeting

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def maximumInvitations(self, favorite):
        """
        :type favorite: List[int]
        :rtype: int
        """
        def find_cycles(adj):
            result = []
            lookup = [False]*len(adj)
            for u in xrange(len(adj)):
                cnt = {}
                while not lookup[u]:
                    lookup[u] = True
                    cnt[u] = len(cnt)
                    u = adj[u]
                if u in cnt:
                    result.append((u, len(cnt)-cnt[u]))
            return result

        def bfs(adj, u, exclude):
            result = 0
            q = [u]
            while q:
                result += 1
                new_q = []
                for u in q:
                    for v in adj[u]:
                        if v == exclude:
                            continue
                        new_q.append(v)
                q = new_q
            return result
            
        inv_adj = [[] for _ in xrange(len(favorite))]  
        for u, v in enumerate(favorite):
            inv_adj[v].append(u)
        cycles = find_cycles(favorite)
        return max(max([l for _, l in cycles if l > 2] or [0]),
                   sum(bfs(inv_adj, u, favorite[u]) + bfs(inv_adj, favorite[u], u) for u, l in cycles if l == 2))

# 2132 Hard 2132 Stamping the Grid

In [ ]:
# Time:  O(m * n)
# Space: O(m * n)

class Solution(object):
    def possibleToStamp(self, grid, stampHeight, stampWidth):
        """
        :type grid: List[List[int]]
        :type stampHeight: int
        :type stampWidth: int
        :rtype: bool
        """
        prefix = [[0]*(len(grid[0])+1) for _ in xrange(len(grid)+1)]
        fit = [[0]*len(grid[0]) for _ in xrange(len(grid))]
        for i in xrange(len(grid)):
            for j in xrange(len(grid[0])):
                prefix[i+1][j+1] = prefix[i+1][j]+prefix[i][j+1]-prefix[i][j]+(1^grid[i][j])
                if i+1 >= stampHeight and j+1 >= stampWidth:
                    x, y = i+1-stampHeight, j+1-stampWidth
                    fit[i][j] = int(prefix[i+1][j+1]-prefix[x][j+1]-prefix[i+1][y]+prefix[x][y] == stampWidth*stampHeight)
        prefix2 = [[0]*(len(grid[0])+1) for _ in xrange(len(grid)+1)]
        for i in xrange(len(grid)):
            for j in xrange(len(grid[0])):
                prefix2[i+1][j+1] = prefix2[i+1][j]+prefix2[i][j+1]-prefix2[i][j]+fit[i][j]
        for i in xrange(len(grid)):
            for j in xrange(len(grid[0])):
                x, y = min(i+stampHeight, len(grid)), min(j+stampWidth, len(grid[0]))
                if not grid[i][j] and not prefix2[x][y]-prefix2[i][y]-prefix2[x][j]+prefix2[i][j]:
                    return False
        return True

# 2136 Hard 2136 Earliest Possible Day of Full Bloom

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

class Solution(object):
    def earliestFullBloom(self, plantTime, growTime):
        """
        :type plantTime: List[int]
        :type growTime: List[int]
        :rtype: int
        """
        order = range(len(growTime))
        order.sort(key=lambda x: growTime[x], reverse=True)
        result = curr = 0
        for i in order:
            curr += plantTime[i]
            result = max(result, curr+growTime[i])
        return result

# 2141 Hard 2141 Maximum Running Time of N Computers

In [ ]:
# Time:  O(nlogm)
# Space: O(1)

import heapq


# greedy
class Solution(object):
    def maxRunTime(self, n, batteries):
        """
        :type n: int
        :type batteries: List[int]
        :rtype: int
        """
        total = sum(batteries)
        for i in xrange(len(batteries)):
            batteries[i] = -batteries[i]  # max_heap
        heapq.heapify(batteries)
        while -batteries[0] > total//n:
            n -= 1
            total -= -heapq.heappop(batteries)
        return total//n


# Time:  O(nlogr), r is the range of possible minutes
# Space: O(1)
# binary search
class Solution2(object):
    def maxRunTime(self, n, batteries):
        """
        :type n: int
        :type batteries: List[int]
        :rtype: int
        """
        def check(n, batteries, x):
            return sum(min(b, x) for b in batteries) >= n*x

        left, right = min(batteries), sum(batteries)//n
        while left <= right:
            mid = left + (right-left)//2
            if not check(n, batteries, mid):
                right = mid-1
            else:
                left = mid+1
        return right

# 2143 Hard 2143 Choose Numbers From Two Arrays in Range

In [ ]:
# Time:  O(n^2 * v), v is max(max(nums1), max(nums2))
# Space: O(n * v)

import collections
import itertools


# dp
class Solution(object):
    def countSubranges(self, nums1, nums2):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :rtype: int
        """
        MOD = 10**9+7

        result = 0
        dp = collections.Counter()
        for x, y in itertools.izip(nums1, nums2):
            new_dp = collections.Counter()
            new_dp[x] += 1
            new_dp[-y] += 1
            for v, c in dp.iteritems():
                new_dp[v+x] = (new_dp[v+x]+c)%MOD
                new_dp[v-y] = (new_dp[v-y]+c)%MOD
            dp = new_dp
            result = (result+dp[0])%MOD
        return result

# 2147 Hard 2147 Number of Ways to Divide a Long Corridor

In [ ]:
# Time:  O(n)
# Space: O(1)

# greedy, combinatorics
class Solution(object):
    def numberOfWays(self, corridor):
        """
        :type corridor: str
        :rtype: int
        """
        MOD = 10**9+7
        result, cnt, j = 1, 0, -1
        for i, x in enumerate(corridor):
            if x != 'S':
                continue
            cnt += 1
            if cnt >= 3 and cnt%2:
                result = result*(i-j)%MOD
            j = i
        return result if cnt and cnt%2 == 0 else 0

# 2151 Hard 2151 Maximum Good People Based on Statements

In [ ]:
# Time:  O(n^2 * 2^n)
# Space: O(1)

# brute force, bitmask
class Solution(object):
    def maximumGood(self, statements):
        """
        :type statements: List[List[int]]
        :rtype: int
        """
        def check(mask):
            return all(((mask>>j)&1) == statements[i][j]
                       for i in xrange(len(statements)) if (mask>>i)&1 
                       for j in xrange(len(statements[i])) if statements[i][j] != 2)

        def popcount(x):
            result = 0
            while x:
                x &= x-1
                result += 1
            return result

        result = 0
        for mask in xrange(1<<len(statements)):
            if check(mask):
                result = max(result, popcount(mask))
        return result

# 2156 Hard 2156 Find Substring With Given Hash Value

In [ ]:
# Time:  O(n)
# Space: O(1)

# rolling hash
class Solution(object):
    def subStrHash(self, s, power, modulo, k, hashValue):
        """
        :type s: str
        :type power: int
        :type modulo: int
        :type k: int
        :type hashValue: int
        :rtype: str
        """
        h, idx = 0, -1
        pw = pow(power, k-1, modulo)
        for i in reversed(xrange(len(s))):
            if i+k < len(s):
                h = (h-(ord(s[i+k])-ord('a')+1)*pw)%modulo
            h = (h*power+(ord(s[i])-ord('a')+1))%modulo
            if h == hashValue:
                idx = i
        return s[idx:idx+k]

# 2157 Hard 2157 Groups of Strings

In [ ]:
# Time:  O(26 * n)
# Space: O(26 * n)

class UnionFind(object):  # Time: O(n * alpha(n)), Space: O(n)
    def __init__(self, n):
        self.set = range(n)
        self.rank = [0]*n
        self.size = [1]*n
        self.total = n

    def find_set(self, x):
        stk = []
        while self.set[x] != x:  # path compression
            stk.append(x)
            x = self.set[x]
        while stk:
            self.set[stk.pop()] = x
        return x

    def union_set(self, x, y):
        x, y = self.find_set(x), self.find_set(y)
        if x == y:
            return False
        if self.rank[x] > self.rank[y]:  # union by rank
            x, y = y, x
        self.set[x] = self.set[y]
        if self.rank[x] == self.rank[y]:
            self.rank[y] += 1
        self.size[y] += self.size[x]
        self.total -= 1
        return True


# bitmasks, union find
class Solution(object):
    def groupStrings(self, words):
        """
        :type words: List[str]
        :rtype: List[int]
        """
        uf = UnionFind(len(words))
        lookup = {}
        for i, x in enumerate(words):
            mask = reduce(lambda x, y: x|(1<<(ord(y)-ord('a'))), x, 0)
            if mask not in lookup:
                lookup[mask] = i
            uf.union_set(i, lookup[mask])
            bit = 1
            while bit <= mask:
                if mask&bit:
                    if mask^bit not in lookup:
                        lookup[mask^bit] = i
                    uf.union_set(i, lookup[mask^bit])
                bit <<= 1
        return [uf.total, max(uf.size)]

# 2158 Hard 2158 Amount of New Area Painted Each Day

In [ ]:
"""
1. Build sorted records = [(position, index, isStart)...]
2. Iterate through all positions and maintain a box with all the "index" of the records its position is in start ~ end
3. The smallest index in the box is the actual one that is paiting.

Time: O(NLogN+P), N is the count of paint. Sorting the records takes NLogN. P is the max position.
Although there is a while loop when iterate through P, each record is only being iterated once.
O(NLogN + P + NLogN) ~= O(NLogN + P)
Space: O(N)
"""
from sortedcontainers import SortedList

class Solution(object):
    def amountPainted(self, paint):
        ans = [0]*len(paint)
        box = SortedList()
        records = []
        maxPos = float('-inf')
        
        #[1]
        for i, (start, end) in enumerate(paint):
            records.append((start, i, -1))
            records.append((end, i, 1))
            maxPos = max(maxPos, end)
        
        records.sort()
        
        #[2]
        i = 0
        for pos in xrange(maxPos+1):
            while i<len(records) and records[i][0]==pos:
                _, index, t = records[i]
                if t==-1:
                    box.add(index)
                else:
                    box.remove(index)
                i += 1
            
            if box: ans[box[0]] += 1 #[3]
        return ans

# 2163 Hard 2163 Minimum Difference in Sums After Removal of Elements

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import heapq


# heap, prefix sum
class Solution(object):
    def minimumDifference(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        max_heap = []
        for i in xrange(len(nums)//3):
            heapq.heappush(max_heap, -nums[i])
        prefix = [0]*(len(nums)//3+1)
        prefix[0] = -sum(max_heap)
        for i in xrange(len(nums)//3):
            x = -heapq.heappushpop(max_heap, -nums[i+len(nums)//3])
            prefix[i+1] = prefix[i]-x+nums[i+len(nums)//3]

        min_heap = []
        for i in reversed(xrange(len(nums)//3*2, len(nums))):
            heapq.heappush(min_heap, nums[i])
        suffix = sum(min_heap)
        result = prefix[len(nums)//3]-suffix
        for i in reversed(xrange(len(nums)//3)):
            x = heapq.heappushpop(min_heap, nums[i+len(nums)//3])
            suffix += -x+nums[i+len(nums)//3]
            result = min(result, prefix[i]-suffix)
        return result

# 2167 Hard 2167 Minimum Time to Remove All Cars Containing Illegal Goods

In [ ]:
# Time:  O(n)
# Space: O(1)

# dp
class Solution(object):
    def minimumTime(self, s):
        """
        :type s: str
        :rtype: int
        """
        left = 0
        result = left+(len(s)-0)
        for i in xrange(1, len(s)+1):
            left = min(left+2*(s[i-1] == '1'), i)
            result = min(result, left+(len(s)-i))
        return result


# Time:  O(n)
# Space: O(n)
# dp
class Solution2(object):
    def minimumTime(self, s):
        """
        :type s: str
        :rtype: int
        """
        result, right = len(s), [0]*(len(s)+1)
        for i in reversed(xrange(len(s))):
            right[i] = min(right[i+1]+2*(s[i] == '1'), len(s)-i)
        left = 0
        result = left+right[0]
        for i in xrange(1, len(s)+1):
            left = min(left+2*(s[i-1] == '1'), i)     
            result = min(result, left+right[i])
        return result

# 2172 Hard 2172 Maximum AND Sum of Array

In [ ]:
# Time:  O(n^3)
# Space: O(n^2)

# weighted bipartite matching solution
class Solution(object):
    def maximumANDSum(self, nums, numSlots):
        """
        :type nums: List[int]
        :type numSlots: int
        :rtype: int
        """
        # Template translated from:
        # https://github.com/kth-competitive-programming/kactl/blob/main/content/graph/WeightedMatching.h
        def hungarian(a):  # Time: O(n^2 * m), Space: O(n + m)
            if not a:
                return 0, []
            n, m = len(a)+1, len(a[0])+1
            u, v, p, ans = [0]*n, [0]*m, [0]*m, [0]*(n-1)
            for i in xrange(1, n):
                p[0] = i
                j0 = 0  # add "dummy" worker 0
                dist, pre = [float("inf")]*m, [-1]*m
                done = [False]*(m+1)
                while True:  # dijkstra
                    done[j0] = True
                    i0, j1, delta = p[j0], None, float("inf")
                    for j in xrange(1, m):
                        if done[j]:
                            continue
                        cur = a[i0-1][j-1]-u[i0]-v[j]
                        if cur < dist[j]:
                            dist[j], pre[j] = cur, j0
                        if dist[j] < delta:
                            delta, j1 = dist[j], j
                    for j in xrange(m):
                        if done[j]:
                            u[p[j]] += delta
                            v[j] -= delta
                        else:
                            dist[j] -= delta
                    j0 = j1
                    if not p[j0]:
                        break
                while j0:  # update alternating path
                    j1 = pre[j0]
                    p[j0], j0 = p[j1], j1
            for j in xrange(1, m):
                if p[j]:
                    ans[p[j]-1] = j-1
            return -v[0], ans  # min cost
    
        return -hungarian([[-((nums[i] if i < len(nums) else 0) & (1+x//2)) for x in xrange(2*numSlots)] for i in xrange(2*numSlots)])[0]


# Time:  O(n^3)
# Space: O(n^2)
from scipy.optimize import linear_sum_assignment as hungarian
import itertools


# 3rd-party weighted bipartite matching solution
class Solution2(object):
    def maximumANDSum(self, nums, numSlots):
        """
        :type nums: List[int]
        :type numSlots: int
        :rtype: int
        """
        adj = [[-((nums[i] if i < len(nums) else 0) & (1+x//2)) for x in xrange(2*numSlots)] for i in xrange(2*numSlots)]
        return -sum(adj[i][j] for i, j in itertools.izip(*hungarian(adj)))    


# Time:  O(n * 3^n)
# Space: O(3^n)
# bottom-up dp (hard to implement but faster)
class Solution3(object):
    def maximumANDSum(self, nums, numSlots):
        """
        :type nums: List[int]
        :type numSlots: int
        :rtype: int
        """
        def count(x):
            result = 0
            while x:
                result += x%3
                x //= 3
            return result

        dp = [0]*(3**numSlots)
        for mask in xrange(1, len(dp)):
            i = count(mask)-1
            x = nums[i] if i < len(nums) else 0
            base = 1
            for slot in xrange(1, numSlots+1):
                if mask//base%3:
                    dp[mask] = max(dp[mask], (x&slot)+dp[mask-base])
                base *= 3
        return dp[-1]


# Time:  O(n * 3^n)
# Space: O(3^n)
# memoization, top-down dp (easy to implement but slower)
class Solution4(object):
    def maximumANDSum(self, nums, numSlots):
        """
        :type nums: List[int]
        :type numSlots: int
        :rtype: int
        """
        def memoiztion(i, mask):  # i is metadata, which could be derived from mask, just for shorter implementation
            if lookup[mask] != -1:
                return lookup[mask]
            x = nums[i] if i < len(nums) else 0
            base = 1
            for slot in xrange(1, numSlots+1):
                if mask//base%3:
                     lookup[mask] = max(lookup[mask], (x&slot)+memoiztion(i-1, mask-base))
                base *= 3
            return lookup[mask]
        
        lookup = [-1]*(3**numSlots)
        lookup[0] = 0
        return memoiztion(2*numSlots-1, 3**numSlots-1)